In [1]:
pwd

'/root/hald_autoupdate/2023_03_13'

In [2]:
import os
import pandas as pd

import csv
import pickle
import json


In [7]:
pd.set_option('display.max_columns', 50)

In [8]:
jcr=pd.read_csv("../JCR/jcr2022.tsv",sep="\t")

In [9]:
len(jcr)

21430

In [10]:
jcr[-10:-1]

,rank,journalTitle,abbrJournal,year,edition,issn,totalCites,journalImpactFactor,impactFactorWithoutJournalSelfCites,fiveYearImpactFactor,immediacyIndex,citableItems,citedHalfLife,citingHalfLife,eigenFactorScore,articleInfluenceScore,citesCurrent,articlesCurrent,selfCites,impactFactorSelfCites,originalResearch,jifPercentile,normEigenFactor
21420,12425,AGENDA,AGENDA,2022,NaN,0002-0796,159,0.0,0.0,0.0,0.0,0,17.8,0.0,0.00005,0.114,NaN,NaN,NaN,NaN,NaN,0.0,0.01206
21421,12425,Abakos,ABAKOS,2022,NaN,2316-9451,7,0.0,0.0,0.0,0.0,18,0.0,10.7,0.00003,0.068,NaN,NaN,NaN,NaN,NaN,0.0,0.00733
21422,12425,Adalya,ADALYA,2022,NaN,1301-2746,75,0.0,0.0,0.0,0.0,19,0.0,16.2,0.00027,0.306,NaN,NaN,NaN,NaN,NaN,0.0,0.05846
21423,12425,Anduli,ANDULI,2022,NaN,1696-0270,35,0.0,0.0,0.0,0.0,15,0.0,14.6,0.00007,0.150,NaN,NaN,NaN,NaN,NaN,0.0,0.01555
21424,12425,Anemia,ANEMIA,2022,NaN,2090-1267,502,0.0,0.0,0.0,0.0,14,7.3,7.7,0.00030,0.550,NaN,NaN,NaN,NaN,NaN,0.0,0.06589
21425,12425,Appeal,APPEAL,2022,NaN,1205-612X,56,0.0,0.0,0.0,0.0,6,0.0,4.8,0.00019,0.550,NaN,NaN,NaN,NaN,NaN,0.0,0.04134
21426,12425,Atenas,ATENAS,2022,NaN,1682-2749,60,0.0,0.0,0.0,0.0,0,0.0,0.0,0.00007,0.034,NaN,NaN,NaN,NaN,NaN,0.0,0.01561
21427,12425,Atenea,ATENEA-CONCEPC,2022,NaN,0718-0462,105,0.0,0.0,0.0,0.0,35,8.8,15.8,0.00034,0.281,NaN,NaN,NaN,NaN,NaN,0.0,0.07463
21428,12425,AusArt,AUSART,2022,NaN,2340-9134,13,0.0,0.0,0.0,0.0,32,0.0,11.8,0.00002,0.017,NaN,NaN,NaN,NaN,NaN,0.0,0.00594


In [11]:
jt2if_all={}
for i in range(len(jcr)):
    if jt2if_all.get(jcr.iloc[i,1].lower()):
        if jt2if_all.get(jcr.iloc[i,1].lower())==0.0:
            jt2if_all[jcr.iloc[i,1].lower()]=jcr.iloc[i,7]
    else:
        jt2if_all[jcr.iloc[i,1].lower()]=jcr.iloc[i,7]
jt2if_all['aging']=jt2if_all.get('aging-us')

In [12]:
jt2if_brief={}
for i in range(len(jcr)):
    if jt2if_brief.get(jcr.iloc[i,2].lower()):
        if jt2if_brief.get(jcr.iloc[i,2].lower())==0.0:
            jt2if_brief[jcr.iloc[i,2].lower()]=jcr.iloc[i,7]
    else:
        jt2if_brief[jcr.iloc[i,2].lower()]=jcr.iloc[i,7]
jt2if_brief['aging']=jt2if_brief.get('aging-us')

In [13]:
list(jt2if_brief.items())[:2]

[('ca-cancer j clin', 286.13), ('lancet', 202.731)]

In [14]:
len(jt2if_brief)

21429

In [15]:
jt2if5_all={}
for i in range(len(jcr)):
    if jt2if5_all.get(jcr.iloc[i,1].lower()):
        if jt2if5_all.get(jcr.iloc[i,1].lower())==0.0:
            jt2if5_all[jcr.iloc[i,1].lower()]=jcr.iloc[i,7]
    else:
        jt2if5_all[jcr.iloc[i,1].lower()]=jcr.iloc[i,9]
jt2if5_all['aging']=jt2if5_all.get('aging-us')

In [16]:
jt2if5_brief={}
for i in range(len(jcr)):
    
    if jt2if5_brief.get(jcr.iloc[i,2].lower()):
        if jt2if5_brief.get(jcr.iloc[i,2].lower())==0.0:
            jt2if5_brief[jcr.iloc[i,2].lower()]=jcr.iloc[i,7]
    else:
        jt2if5_brief[jcr.iloc[i,2].lower()]=jcr.iloc[i,9]
jt2if5_brief['aging']=jt2if5_brief.get('aging-us')

In [17]:
list(jt2if5_brief.items())[:2]

[('ca-cancer j clin', 334.259), ('lancet', 130.838)]

In [18]:
import pymongo 

In [19]:
from pymongo import MongoClient

In [20]:
client = MongoClient(username="hald",password="XXX")

In [21]:
db = client.hald

In [22]:
collections_pubtator_info = db.pubtator_info

In [23]:
pubtator_info=list(collections_pubtator_info.find())

In [24]:
len(pubtator_info)

173787

In [25]:
pubtator_info[10]

{'_id': ObjectId('6393478a5303aaecac2745e7'),
 'PMID': '35794516',
 'OWN': 'NLM',
 'STAT': 'MEDLINE',
 'DCOM': '20220708',
 'LR': '20220708',
 'IS': '1471-227X (Electronic) 1471-227X (Linking)',
 'VI': '22',
 'IP': '1',
 'DP': '2022 Jul 6',
 'TI': 'Long-term prognosis and clinical course of choking-induced cardiac arrest in patients without the return of spontaneous circulation at hospital arrival: a population-based community study from the Shizuoka Kokuho Database.',
 'PG': '120',
 'LID': '10.1186/s12873-022-00676-8 [doi]',
 'AB': 'BACKGROUND: The risk of choking increases with aging, and the number of cases of choking-induced cardiac arrest is increasing. However, few studies have examined the prognosis of choking-induced cardiac arrest. The aim of this study was to reveal the rates of survival and dependence on devices in the long term after choking-induced cardiac arrest. METHODS: We analyzed data from the Shizuoka Kokuho Database, which consists of claims data of approximately 2.

In [26]:
pubtator_info_dict={}
for i in pubtator_info:
    pubtator_info_dict[i.get('PMID')]=i

In [27]:
collections_Relation_final=db.Relation_final

In [28]:
triplelist=list(collections_Relation_final.find())

In [29]:
triplelist[0]

{'_id': ObjectId('641143e7ee32117b2d89852f'),
 'PMID': '35926182',
 'entity1': 'SF3B1',
 'entity2': 'EIF2AK1',
 'sentence': 'Here, using single-cell technologies and functional validation studies of primary SF3B1-mutant MDS-RS samples, we show that SF3B1 mutations lead to the activation of the EIF2AK1 pathway in response to heme deficiency and that targeting this pathway rescues aberrant erythroid differentiation and enables the red blood cell maturation of MDS-RS erythroblasts.',
 'AB': 'SF3B1 mutations, which occur in 20% of patients with myelodysplastic syndromes (MDS), are the hallmarks of a specific MDS subtype, MDS with ringed sideroblasts (MDS-RS), which is characterized by the accumulation of erythroid precursors in the bone marrow and primarily affects the elderly population. Here, using single-cell technologies and functional validation studies of primary SF3B1-mutant MDS-RS samples, we show that SF3B1 mutations lead to the activation of the EIF2AK1 pathway in response to hem

In [30]:
# 124608
len(triplelist)

124608

In [31]:
be=['am','is','are','was','were','been','have been','has been']
bedict={}
for i in be:
    bedict[i]='be'


In [32]:
'''将entity_one_transformed和entity_two_transformed的列表外套脱掉，筛掉同篇文献同一个实体对，筛掉两个单词为同一实体'''
import copy
triplelist_takeoff=[]
transformed_triples=set()
box=[]
box2=[]
acount=0
for i in triplelist:
    maxlength=[]
    allennlp_list=[]
    if i.get('openie_final'):
        maxlength=i.get('openie_final')[0]
        for j in i.get('openie_final'):
            if len(j[1])>len(maxlength[1]):
                maxlength=j
        mid_list=maxlength[1].split(' ')
        for p,q in enumerate(mid_list):
            if bedict.get(q):
                mid_list[p]='be'
        maxlength[1]=' '.join(mid_list)
        for k in maxlength[1]:
            if k in maxlength[0].split(' ') or k in maxlength[2].split(' '):
                break
        else:
            mid=tuple([maxlength[0],maxlength[1].lower(),maxlength[2],i.get('PMID')])
            if mid not in transformed_triples:
                copydict={'triple':maxlength,'PMID':i.get('PMID'),'sentence':i.get('sentence'),
                          'AB':i.get('AB'),'entity_one':i.get('target1'),'entity_one_type':i.get('target1_type'),
                          'entity_two':i.get('target2'),'entity_two_type':i.get('target2_type'),
                          'entity_one_transformed':i.get('entity1'),'entity_two_transformed':i.get('entity2'),
                            'transformed_triple':mid}
                triplelist_takeoff.append(copydict)
                transformed_triples.add(mid)
            if maxlength[1]=='associated' or maxlength[1]=='associate' :
                mid=tuple([maxlength[2],maxlength[1].lower(),maxlength[0],i.get('PMID')])
                if mid not in transformed_triples:
                    copydict={'triple':maxlength,'PMID':i.get('PMID'),'sentence':i.get('sentence'),
                              'AB':i.get('AB'),'entity_one':i.get('target2'),'entity_one_type':i.get('target2_type'),
                              'entity_two':i.get('target1'),'entity_two_type':i.get('target1_type'),
                              'entity_one_transformed':i.get('entity2'),'entity_two_transformed':i.get('entity1'),
                                'transformed_triple':mid}
                    triplelist_takeoff.append(copydict)
                    transformed_triples.add(mid)
        for j in i.get('openie_final'):
            if j==maxlength:
                continue
            judge=True
            for k in j[1].split(' '):
                if k not in maxlength[1].split(' '):
                    judge=False
                    break
            if not judge:
                mid_list=j[1].split(' ')
                for p,q in enumerate(mid_list):
                    if bedict.get(q):
                        mid_list[p]='be'
                j[1]=' '.join(mid_list)
                mid=tuple([j[0],j[1].lower(),j[2],i.get('PMID')])
                for k in j[1]:
                    if k in j[0].split(" ") or k in j[1].split(" "):
                        break
                else:
                    if mid not in transformed_triples:
                        copydict={'triple':j,'PMID':i.get('PMID'),'sentence':i.get('sentence'),
                                  'AB':i.get('AB'),'entity_one':i.get('target1'),'entity_one_type':i.get('target1_type'),
                                  'entity_two':i.get('target2'),'entity_two_type':i.get('target2_type'),
                                  'entity_one_transformed':i.get('entity1'),'entity_two_transformed':i.get('entity2'),
                                    'transformed_triple':mid}
                        triplelist_takeoff.append(copydict)
                        transformed_triples.add(mid)
                    if j[1]=='associated' or j[1]=='associate' :
                        mid=tuple([j[2],j[1].lower(),j[0],i.get('PMID')])
                        if mid not in transformed_triples:
                            copydict={'triple':j,'PMID':i.get('PMID'),'sentence':i.get('sentence'),
                                      'AB':i.get('AB'),'entity_one':i.get('target2'),'entity_one_type':i.get('target2_type'),
                                      'entity_two':i.get('target1'),'entity_two_type':i.get('target1_type'),
                                      'entity_one_transformed':i.get('entity2'),'entity_two_transformed':i.get('entity1'),
                                        'transformed_triple':mid}
                            triplelist_takeoff.append(copydict)
                            transformed_triples.add(mid)
    if i.get('allennlp_final'):
        for j in i.get('allennlp_final'):
            if len(maxlength)>0 and j[1] in maxlength[1].split(' '):
                pass
            else:
                mid=tuple([j[0],j[1].lower(),j[2],i.get('PMID')])
                if j[1] in j[0].split(' ') or j[1] in j[2].split(' '):
                    continue
                if mid not in transformed_triples:
                    allennlp_list.append((j[0],j[2]))
                    copydict={'triple':j,'PMID':i.get('PMID'),'sentence':i.get('sentence'),
                              'AB':i.get('AB'),'entity_one':i.get('target1'),'entity_one_type':i.get('target1_type'),
                              'entity_two':i.get('target2'),'entity_two_type':i.get('target2_type'),
                              'entity_one_transformed':i.get('entity1'),'entity_two_transformed':i.get('entity2'),
                                'transformed_triple':mid}
                    triplelist_takeoff.append(copydict)
                    transformed_triples.add(mid)
                else:
                    box2.append(i)
                if j[1]=='associated' or j[1]=='associate' :
                    mid=tuple([j[2],j[1].lower(),j[0],i.get('PMID')])
                    if mid not in transformed_triples:
                        allennlp_list.append([j[2],j[0]])
                        copydict={'triple':j,'PMID':i.get('PMID'),'sentence':i.get('sentence'),
                                  'AB':i.get('AB'),'entity_one':i.get('target2'),'entity_one_type':i.get('target2_type'),
                                  'entity_two':i.get('target1'),'entity_two_type':i.get('target1_type'),
                                  'entity_one_transformed':i.get('entity2'),'entity_two_transformed':i.get('entity1'),
                                    'transformed_triple':mid}
                        triplelist_takeoff.append(copydict)
                        transformed_triples.add(mid)
                    else:
                        box2.append(i)
    if i.get('networkx'):
        if len(maxlength)>0 and i.get('networkx')[1] in maxlength[1].split(' '):
            box.append(i)
            pass
        else:
            # for j in allennlp_list:
            #     if i.get('networkx')[0]==j[1] and i.get('networkx')[1]==j[0]:
            #         print(i.get('networkx'))
            #         print(i)
            if i.get('networkx')[1] in i.get('networkx')[0].split(' ') or i.get('networkx')[1] in i.get('networkx')[2].split(' '):
                acount+=1
                continue
            mid=tuple([i.get('networkx')[0],i.get('networkx')[1].lower(),i.get('networkx')[2],i.get('PMID')])
            if mid not in transformed_triples:
                copydict={'triple':i.get('networkx'),'PMID':i.get('PMID'),'sentence':i.get('sentence'),
                          'AB':i.get('AB'),'entity_one':i.get('target1'),'entity_one_type':i.get('target1_type'),
                          'entity_two':i.get('target2'),'entity_two_type':i.get('target2_type'),
                          'entity_one_transformed':i.get('entity1'),'entity_two_transformed':i.get('entity2'),
                            'transformed_triple':mid}
                triplelist_takeoff.append(copydict)
                transformed_triples.add(mid)
            if i.get('networkx')[1]=='associated' or i.get('networkx')[1]=='associate' :
                mid=tuple([i.get('networkx')[2],i.get('networkx')[1].lower(),i.get('networkx')[0],i.get('PMID')])
                if mid not in transformed_triples:
                    copydict={'triple':i.get('networkx'),'PMID':i.get('PMID'),'sentence':i.get('sentence'),
                              'AB':i.get('AB'),'entity_one':i.get('target2'),'entity_one_type':i.get('target2_type'),
                              'entity_two':i.get('target1'),'entity_two_type':i.get('target1_type'),
                              'entity_one_transformed':i.get('entity2'),'entity_two_transformed':i.get('entity1'),
                                'transformed_triple':mid}
                    triplelist_takeoff.append(copydict)
                    transformed_triples.add(mid)



In [33]:
len(box)

1386

In [34]:
len(box2)

1843

In [35]:
mid

('rs11189312', 'related', 'sarcopenia', '36573383')

In [36]:
print(len(triplelist_takeoff))
print(len(transformed_triples))

145434
145434


In [37]:
triplelist_takeoff[0]

{'triple': ['SF3B1', 'lead', 'EIF2AK1'],
 'PMID': '35926182',
 'sentence': 'Here, using single-cell technologies and functional validation studies of primary SF3B1-mutant MDS-RS samples, we show that SF3B1 mutations lead to the activation of the EIF2AK1 pathway in response to heme deficiency and that targeting this pathway rescues aberrant erythroid differentiation and enables the red blood cell maturation of MDS-RS erythroblasts.',
 'AB': 'SF3B1 mutations, which occur in 20% of patients with myelodysplastic syndromes (MDS), are the hallmarks of a specific MDS subtype, MDS with ringed sideroblasts (MDS-RS), which is characterized by the accumulation of erythroid precursors in the bone marrow and primarily affects the elderly population. Here, using single-cell technologies and functional validation studies of primary SF3B1-mutant MDS-RS samples, we show that SF3B1 mutations lead to the activation of the EIF2AK1 pathway in response to heme deficiency and that targeting this pathway resc

In [38]:
triplelist_takeoff_dict={}
for i in triplelist_takeoff:
    triplelist_takeoff_dict[(i.get('entity_one_transformed'),i.get('triple')[1],i.get('entity_two_transformed'),i.get('PMID'))]=triplelist_takeoff_dict.get((i.get('entity_one_transformed'),i.get('entity_two_transformed'),i.get('PMID')),[])+[i]

In [39]:
len(triplelist_takeoff_dict)

134091

In [40]:
triplelist_takeoffs=[]
a=[]
b=[]
c=[]
d=[]
for i,j in triplelist_takeoff_dict.items():
    if len(j)==1:
        triplelist_takeoffs.append(j[0])
    else:
        deprecated_set=set()
        for k in j:
            if k.get('entity_one').lower()==k.get('entity_one_transformed').lower() and k.get('entity_two').lower()==k.get('entity_two_transformed').lower():
                triplelist_takeoffs.append(k)
                a.append(k)
                break
        else:
            for k in j:
                if k.get('entity_one').lower()==k.get('entity_one_transformed').lower():
                    triplelist_takeoffs.append(k)
                    b.append(k)
                    break
            else:
                for k in j:
                    if k.get('entity_two').lower()==k.get('entity_two_transformed').lower():
                        c.append(k)
                        triplelist_takeoffs.append(k)
                        break
                else:
                    d.append(k)
                    triplelist_takeoffs.append(j[0])

In [41]:
names=locals()
for i in ['a','b','c','d']:
    print(len(names[i]))

0
0
0
0


In [42]:
len(triplelist_takeoffs)

134091

In [43]:
# 114062
len(triplelist_takeoff)

145434

In [44]:
import re
triplelist_takesoff=[]
for i in triplelist_takeoffs:
    if not re.search(r'\`|\~|\!|\@|\#|\$|\%|\^|\&|\*|\(|\)|\_|\+|\=|\{|\[|\}|\]|\;|\:|\'|\"|\,|\<|\.|\>|\/|\?',i.get('triple')[1]):
        if not re.search(r'(\s\d+\s)|(\b\d+\b)',i.get('triple')[1]): 
            triplelist_takesoff.append(i)  
        

In [45]:
triplelistnewcopy=copy.deepcopy(triplelist_takesoff)

In [46]:
deprecated_relations=pd.read_csv('../classofrelations/deprecated.csv',header=None,names=['word'])

In [47]:
len(deprecated_relations)

209

In [48]:
deprecated_relations.head()

,word
0,% for
1,'s III be
2,)evaluate
3,#NAME?
4,#NAME?


In [49]:
deprecated_relations_dict={}
for i in deprecated_relations['word'].to_list():
    deprecated_relations_dict[i.lower()]=True

In [50]:
deprecated_relations_dict.get('have')

True

In [51]:
list(deprecated_relations_dict.items())[20]

('aforemention', True)

In [52]:
triplelistnewcopy[0]

{'triple': ['SF3B1', 'lead', 'EIF2AK1'],
 'PMID': '35926182',
 'sentence': 'Here, using single-cell technologies and functional validation studies of primary SF3B1-mutant MDS-RS samples, we show that SF3B1 mutations lead to the activation of the EIF2AK1 pathway in response to heme deficiency and that targeting this pathway rescues aberrant erythroid differentiation and enables the red blood cell maturation of MDS-RS erythroblasts.',
 'AB': 'SF3B1 mutations, which occur in 20% of patients with myelodysplastic syndromes (MDS), are the hallmarks of a specific MDS subtype, MDS with ringed sideroblasts (MDS-RS), which is characterized by the accumulation of erythroid precursors in the bone marrow and primarily affects the elderly population. Here, using single-cell technologies and functional validation studies of primary SF3B1-mutant MDS-RS samples, we show that SF3B1 mutations lead to the activation of the EIF2AK1 pathway in response to heme deficiency and that targeting this pathway resc

In [53]:
i

'promising'

In [54]:
bedict

{'am': 'be',
 'is': 'be',
 'are': 'be',
 'was': 'be',
 'were': 'be',
 'been': 'be',
 'have been': 'be',
 'has been': 'be'}

In [55]:
not_relations=pd.read_csv('../classofrelations/not_relations.csv',header=None,names=['word'])

In [56]:
len(not_relations)

61

In [57]:
not_relations.head()

,word
0,1p13 near
1,a of
2,abstractdiagnosing
3,acetylcholinesterase
4,apolipoprotein


In [58]:
not_relations_dict={}
for i in not_relations['word'].to_list():
    not_relations_dict[i.lower()]=True

In [59]:
list(not_relations_dict.items())[20]

('near 7q11 be', True)

In [60]:
bedict

{'am': 'be',
 'is': 'be',
 'are': 'be',
 'was': 'be',
 'were': 'be',
 'been': 'be',
 'have been': 'be',
 'has been': 'be'}

In [61]:
triplelistnew=[]
for i in triplelistnewcopy: 
    if not deprecated_relations_dict.get(i.get('triple')[1].lower()) and not not_relations_dict.get(i.get('triple')[1].lower()):
        if i.get('triple')[1][0]=='\-':
            continue
        temp_triple=i.get('triple')[1].split(' ')
        for j,k in enumerate(temp_triple):
            if bedict.get(k.lower()):
                temp_triple[j]=bedict.get(k.lower())
        i['triple'][1]=' '.join(temp_triple)
        if i.get('triple')[1].lower()=='be' or bedict.get(i.get('triple')[1].lower()):
            continue
        if i.get('triple')[1].lower()=='means':
            i['triple'][1]='mean'
        if i.get('triple')[1].lower()=='meets':
            i['triple'][1]='meet'
        
        mid_judge=True
        for j in i.get('triple')[1].split(' '):
            if j.lower() not in i.get('triple')[0].lower().split(' ') and j.lower() not in i.get('triple')[2].lower().split(' '):
                pass
            else:
                mid_judge=False
        if mid_judge:
            triplelistnew.append(i)


In [62]:
#124853
len(triplelistnew)

124471

In [63]:
'''
含有重复的triplelistnew数量为56723
'''

'\n含有重复的triplelistnew数量为56723\n'

In [64]:
triplelistnew[10]

{'triple': ['rs1001179', 'investigated', 'cataract'],
 'PMID': '35556207',
 'sentence': 'Association of SOD1 50 bp Indel, GPX1 (rs1800668), and CAT (rs1001179) genotypes with all clinical variables was investigated in cataract groups by regression statistics in SPSS  16.0.',
 'AB': 'Hyperglycemia plays a pronounced role in accelerating the process of aging due to high oxidative stress which triggers dyslipidemia and subsequently led to the progression of cataract. The aim of this study was to investigate lipid profile and its relationship with genotypes of SOD1, GPX1, and CAT variants in cataract patients. Total n = 680 samples were screened in four groups: senile cataract (SC), diabetic cataract (DC), type 2 diabetes mellitus (DM), and controls (CL). Lipid profile was estimated and compared between groups, and its correlation was tested with glycemic markers. Association of SOD1 50 bp Indel, GPX1 (rs1800668), and CAT (rs1001179) genotypes with all clinical variables was investigated i

In [65]:
triplelistnew[0]

{'triple': ['SF3B1', 'lead', 'EIF2AK1'],
 'PMID': '35926182',
 'sentence': 'Here, using single-cell technologies and functional validation studies of primary SF3B1-mutant MDS-RS samples, we show that SF3B1 mutations lead to the activation of the EIF2AK1 pathway in response to heme deficiency and that targeting this pathway rescues aberrant erythroid differentiation and enables the red blood cell maturation of MDS-RS erythroblasts.',
 'AB': 'SF3B1 mutations, which occur in 20% of patients with myelodysplastic syndromes (MDS), are the hallmarks of a specific MDS subtype, MDS with ringed sideroblasts (MDS-RS), which is characterized by the accumulation of erythroid precursors in the bone marrow and primarily affects the elderly population. Here, using single-cell technologies and functional validation studies of primary SF3B1-mutant MDS-RS samples, we show that SF3B1 mutations lead to the activation of the EIF2AK1 pathway in response to heme deficiency and that targeting this pathway resc

In [66]:
entity_set=set()
for i in triplelistnew:
    entity_set.add(i.get('entity_one_transformed'))
    entity_set.add(i.get('entity_two_transformed'))

In [67]:
len(entity_set)

6254

In [68]:
'''
整合所有实体可能存在的类型 
entitydict就是实体转类型（分号相隔）
'''
import re
entitydict={}
for i,j in enumerate(triplelist):
    if j.get('entity1') in entity_set and j.get('entity2') in entity_set:
        # if j.get('entity_one')==j.get('entity_two'):
        #     continue
        if not entitydict.get(j.get('entity1')) and not entitydict.get(j.get('entity2')):
            entitydict[j.get('entity1')]=j.get('target1_type')
            entitydict[j.get('entity2')]=j.get('target2_type')
        else:
            if entitydict.get(j.get('entity1')):
                if re.search(j.get('target1_type'), entitydict.get(j.get('entity1'))) :
                    pass
                else:
                    entitydict[j.get('entity1')]+=";"+j.get('target1_type')
            else:
                entitydict[j.get('entity1')]=j.get('target1_type')
            if entitydict.get(j.get('entity2')):
                if re.search(j.get('target2_type'), entitydict.get(j.get('entity2'))) :
                    pass
                else:
                    entitydict[j.get('entity2')]+=";"+j.get('target2_type')
            else:
                entitydict[j.get('entity2')]=j.get('target2_type')

In [69]:
len(entitydict)

6254

In [70]:
'''
整合所有实体可能存在的类型 
entitydict就是实体转类型（分号相隔）
'''
import re
entitydict_tolist={}
for i,j in enumerate(triplelist):
    if j.get('entity1') in entity_set and j.get('entity2') in entity_set:
        # if j.get('entity_one')==j.get('entity_two'):
        #     continue
        if not entitydict_tolist.get(j.get('entity1')) and not entitydict_tolist.get(j.get('entity2')):
            entitydict_tolist[j.get('entity1')]=[j.get('target1_type')]
            entitydict_tolist[j.get('entity2')]=[j.get('target2_type')]
        else:
            if entitydict_tolist.get(j.get('entity1')):
                if j.get('target1_type') in entitydict_tolist.get(j.get('entity1')):
                    pass
                else:
                    entitydict_tolist[j.get('entity1')]+=[j.get('target1_type')]
            else:
                entitydict_tolist[j.get('entity1')]=[j.get('target1_type')]
            if entitydict_tolist.get(j.get('entity2')):
                if j.get('target2_type') in  entitydict_tolist.get(j.get('entity2')) :
                    pass
                else:
                    entitydict_tolist[j.get('entity2')]+=[j.get('target2_type')]
            else:
                entitydict_tolist[j.get('entity2')]=[j.get('target2_type')]

In [71]:
entitydict.get('Lipopolysaccharides')

'Carbohydrate;Lipid;Toxin'

In [72]:
entitydict_tolist.get('Lipopolysaccharides')

['Carbohydrate', 'Lipid', 'Toxin']

In [73]:
# 5862
len(entitydict)

6254

In [74]:
list(entitydict.items())[-30:-1]

[('LBR', 'Gene'),
 ('EIF5A2', 'Gene'),
 ('RHOXF1', 'Gene'),
 ('HECTD1', 'Gene'),
 ('RMDN3', 'Gene'),
 ('VAPB', 'Gene'),
 ('PFKP', 'Gene'),
 ('rs11189312', 'Mutation'),
 ('rs10882883', 'Mutation'),
 ('rs17108378', 'Mutation'),
 ('rs35077384', 'Mutation'),
 ('Hereditary Sensory and Motor Neuropathy', 'Disease'),
 ('Vaginal Neoplasms', 'Disease'),
 ('Trichiasis', 'Disease'),
 ('Autoimmune Lymphoproliferative Syndrome', 'Disease'),
 ('Influenza, Human', 'Disease'),
 ('Dental Pulp Necrosis', 'Disease'),
 ('Dacryocystitis', 'Disease'),
 ('Laryngeal Diseases', 'Disease'),
 ('Diastasis, Bone', 'Disease'),
 ('rs4293393', 'Mutation'),
 ('rs10107605', 'Mutation'),
 ('DGKB', 'Gene'),
 ('rs12666606', 'Mutation'),
 ('MYO1B', 'Gene'),
 ('ACKR1', 'Gene'),
 ('LGI4', 'Gene'),
 ('ISLR2', 'Gene'),
 ('Cardiolipins', 'Lipid')]

In [75]:
for i,j in entitydict.items():
    if re.search(';',j):
        print({i:j})


{'Ceramides': 'Carbohydrate;Lipid'}
{'Bleomycin': 'Carbohydrate;Peptide'}
{'Angiotensins': 'Peptide;Protein'}
{'Lipopolysaccharides': 'Carbohydrate;Lipid;Toxin'}
{'Psychosine': 'Carbohydrate;Lipid'}
{'Glycolipids': 'Carbohydrate;Lipid'}
{'Abciximab': 'Peptide;Protein'}
{'Gangliosides': 'Carbohydrate;Lipid'}
{'G(M1) Ganglioside': 'Carbohydrate;Lipid'}
{'G(M3) Ganglioside': 'Carbohydrate;Lipid'}
{'Sphingomyelins': 'Carbohydrate;Lipid'}
{'Glycosphingolipids': 'Carbohydrate;Lipid'}
{'Phospholipid Ethers': 'Carbohydrate;Lipid'}
{'Vancomycin': 'Carbohydrate;Peptide'}
{'Daptomycin': 'Lipid;Peptide'}
{'Glycopeptides': 'Carbohydrate;Peptide'}
{'Ristocetin': 'Carbohydrate;Peptide'}
{'Cilastatin, Imipenem Drug Combination': 'Lipid;Pharmaceutical Preparations'}
{'Phalloidine': 'Peptide;Toxin'}
{'Globosides': 'Carbohydrate;Lipid'}
{'Glucosylceramides': 'Carbohydrate;Lipid'}
{'Cerebrosides': 'Carbohydrate;Lipid'}
{'Sulfoglycosphingolipids': 'Carbohydrate;Lipid'}
{'Glycosylphosphatidylinositols': 'Ca

In [76]:
'''标注entity序号ID'''
entitynum={}
num=1
for i in entitydict.keys():
    entitynum[i]=num
    num+=1

In [77]:
entitynum.get('APOE')

867

In [78]:
for i in triplelistnew:
    i['verb']=i.get('triple')[1]

In [79]:
'''分别计算entity出现次数，relation出现次数和triple出现次数'''
entity_countnum={}
relation_countnum={}
tripleWeight={}
tripleWeightunique=set()
for i in triplelistnew:
    if i.get('entity_one_transformed')!=i.get('entity_two_transformed'):
        if (i.get('entity_one_transformed'),i.get('triple')[1].lower(),i.get('entity_two_transformed'),i.get('PMID')) not in tripleWeightunique:
            tripleWeightunique.add((i.get('entity_one_transformed'),i.get('triple')[1].lower(),i.get('entity_two_transformed'),i.get('PMID')))
            entity_countnum[i.get('entity_one_transformed')]=entity_countnum.get(i.get('entity_one_transformed'),0)+1
            entity_countnum[i.get('entity_two_transformed')]=entity_countnum.get(i.get('entity_two_transformed'),0)+1
            relation_countnum[i.get('triple')[1]]=relation_countnum.get(i.get('triple')[1],0)+1
            tripleWeight[(i.get('entity_one_transformed'),i.get('triple')[1].lower(), i.get('entity_two_transformed'))]=tripleWeight.get((i.get('entity_one_transformed'),i.get('triple')[1].lower(), i.get('entity_two_transformed')),0)+1

In [80]:
len(entity_countnum)

6254

In [81]:
# 51551
len(tripleWeight)

100255

In [82]:
len(triplelistnew)

124471

In [83]:
triplelistnew[0]

{'triple': ['SF3B1', 'lead', 'EIF2AK1'],
 'PMID': '35926182',
 'sentence': 'Here, using single-cell technologies and functional validation studies of primary SF3B1-mutant MDS-RS samples, we show that SF3B1 mutations lead to the activation of the EIF2AK1 pathway in response to heme deficiency and that targeting this pathway rescues aberrant erythroid differentiation and enables the red blood cell maturation of MDS-RS erythroblasts.',
 'AB': 'SF3B1 mutations, which occur in 20% of patients with myelodysplastic syndromes (MDS), are the hallmarks of a specific MDS subtype, MDS with ringed sideroblasts (MDS-RS), which is characterized by the accumulation of erythroid precursors in the bone marrow and primarily affects the elderly population. Here, using single-cell technologies and functional validation studies of primary SF3B1-mutant MDS-RS samples, we show that SF3B1 mutations lead to the activation of the EIF2AK1 pathway in response to heme deficiency and that targeting this pathway resc

In [84]:
import time
import datetime
import re
dropout=[]
months=['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']
import re
def cal_date(i):
    j=i
    if re.search('[S,s]upplement',j,flags=re.M|re.I):
        j=re.sub('[S,s]upplement.*','',j,flags=re.M|re.I)
    j=re.sub('\s?-\s?','-',j)
    j=re.sub('\.','',j)
    j=re.sub('Aout','Aug',j)
    if len(j.split(' '))==3:
        try:
            date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y %b %d'), '%Y%m%d'))
        except:
            if j.split()[1] in months:
                if re.search(r'\d+',j.split()[2],flags=re.M|re.I):
                    j=j.split()[0]+' '+j.split()[1]+' '+ re.search(r'\d+',j.split()[2],flags=re.M|re.I).group()
                    try:
                        date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y %b %d'), '%Y%m%d'))
                    except:
                        j=j[:4]
                        date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y'), '%Y'+'0101'))
                        print('***************************************ERROR**********************************************')
                else:
                    j=j.split()[0]+' '+j.split()[1]+' '+ '1'
                    try:
                        date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y %b %d'), '%Y%m%d'))
                    except:
                        print(j)
                        j=j[:4]
                        date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y'), '%Y'+'0101'))
            elif j.split()[1] not in months:
                if re.search(r'([a-zA-Z]+)[-/]([a-zA-Z]+)',j.split()[1],flags=re.M|re.I) or re.search(r'([a-zA-Z]+)[-/](\d+)',j.split()[1],flags=re.M|re.I):
                    if re.search(r'([a-zA-Z]+)[-/]([a-zA-Z]+)',j.split()[1],flags=re.M|re.I):
                        middle=re.search(r'([a-zA-Z]+)[-/]([a-zA-Z]+)',j.split()[1],flags=re.M|re.I).group(1)
                    else:
                        middle=re.search(r'([a-zA-Z]+)[-/](\d+)',j.split()[1],flags=re.M|re.I).group(1)
                    if middle in months:
                        if re.search(r'\d+',j.split()[2],flags=re.M|re.I):
                            j=j.split()[0]+' '+middle+' '+ re.search(r'\d+',j.split()[2],flags=re.M|re.I).group()
                        else:
                            j=j.split()[0]+' '+middle+' '+ '1'
                    elif re.match(r'Sum(mer)?',middle):
                        if re.search(r'\d+',j.split()[2],flags=re.M|re.I):
                            j=j.split()[0]+' '+'Jun'+' '+ re.search(r'\d+',j.split()[2],flags=re.M|re.I).group()
                        else:
                            j=j.split()[0]+' '+'Jun'+' '+ '1'
                    elif re.match(r'Spr(ing)?',middle):
                        if re.search(r'\d+',j.split()[2],flags=re.M|re.I):
                            j=j.split()[0]+' '+'Mar'+' '+ re.search(r'\d+',j.split()[2],flags=re.M|re.I).group()
                        else:
                            j=j.split()[0]+' '+'Mar'+' '+ '1'
                    elif re.match(r'Win(ter)?',middle):
                        if re.search(r'\d+',j.split()[2],flags=re.M|re.I):
                            j=j.split()[0]+' '+'Dec'+' '+ re.search(r'\d+',j.split()[2],flags=re.M|re.I).group()
                        else:
                            j=j.split()[0]+' '+'Dec'+' '+ '1'    
                    elif re.match(r'Aut(umn)?',middle) or \
                    re.match(r'Fall',middle):
                        if re.search(r'\d+',j.split()[2],flags=re.M|re.I):
                            j=j.split()[0]+' '+'Sep'+' '+ re.search(r'\d+',j.split()[2],flags=re.M|re.I).group()
                        else:
                            j=j.split()[0]+' '+'Sep'+' '+ '1'      
                    try:
                        date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y %b %d'), '%Y%m%d'))
                    except:
                        print(j)
                        j=j[:4]
                        date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y'), '%Y'+'0101'))
                else:
                    j=j[:4]
                    date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y'), '%Y'+'0101'))
    elif len(j.split())==2:
        if re.match(r'Sum(mer)?',j.split()[1]):
            date=int(j[:4]+'0601')
        elif re.match(r'Spr(ing)?',j.split()[1]):
            date=int(j[:4]+'0301')
        elif re.match(r'Win(ter)?',j.split()[1]):
            date=int(j[:4]+'1201')
        elif re.match(r'Aut(umn)?',j.split()[1]) or re.match(r'Fall',j.split()[1]):
            date=int(j[:4]+'0901')
        elif re.match(r'Sum(mer)?',j.split()[0]):
            date=int(j[-4:]+'0601')
        elif re.match(r'Spr(ing)?',j.split()[0]):
            date=int(j[-4:]+'0301')
        elif re.match(r'Win(ter)?',j.split()[0]):
            date=int(j[-4:]+'1201')
        elif re.match(r'Aut(umn)?',j.split()[0]) or re.match(r'Fall',j.split()[0]):
            date=int(j[-4:]+'0901')
        else:
            try:
                date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y %b'), '%Y%m'+'01'))
            except:
                if re.search(r'([a-zA-Z]+)[-/]([a-zA-Z]+)',j.split()[1],flags=re.M|re.I) or re.search(r'([a-zA-Z]+)[-/](\d+)',j.split()[1],flags=re.M|re.I):
                    if re.search(r'([a-zA-Z]+)[-/]([a-zA-Z]+)',j.split()[1],flags=re.M|re.I):
                        middle=re.search(r'([a-zA-Z]+)[-/]([a-zA-Z]+)',j.split()[1],flags=re.M|re.I).group(1)
                    else:
                        middle=re.search(r'([a-zA-Z]+)[-/](\d+)',j.split()[1],flags=re.M|re.I).group(1)
                    if middle in months:
                        j=j.split()[0]+' '+middle+' '+ '1'
                    elif re.match(r'Sum(mer)?',middle):
                        j=j.split()[0]+' '+'Jun'+' '+ '1'
                    elif re.match(r'Spr(ing)?',middle):
                        j=j.split()[0]+' '+'Mar'+' '+ '1'
                    elif re.match(r'Win(ter)?',middle):
                        j=j.split()[0]+' '+'Dec'+' '+ '1'    
                    elif re.match(r'Aut(umn)?',middle) or \
                    re.match(r'Fall',middle):
                        j=j.split()[0]+' '+'Sep'+' '+ '1'      
                    try:
                        date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y %b %d'), '%Y%m%d'))
                    except:
                        j=j[:4]
                        date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y'), '%Y'+'0101'))
                elif re.search(r'(\d+)[-/](\d+)',j.split()[1],flags=re.M|re.I):
                    midd=re.search(r'(\d+)[-/](\d+)',j.split()[1],flags=re.M|re.I).group(1)
                    if len(midd)!=2:
                        midd='0'+midd
                    j=j[:4]
                    date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y'), '%Y'+midd+'01'))
                else:
                    print(j)
                    j=j[:4]
                    date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y'), '%Y'+'0101'))
    elif len(j.split())==1:
        j=j[:4]
        date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y'), '%Y'+'0101'))
    else:
        j=j.split()[0]+' '+j.split()[1]+' '+re.search(r'\d+',j.split()[2],flags=re.M|re.I).group()
        date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y %b %d'), '%Y%m%d'))
    return date


In [85]:
for i in triplelistnew:
    if jt2if_brief.get(pubtator_info_dict.get(i.get('PMID')).get('TA').lower()):
        i['IF']=jt2if_brief.get(pubtator_info_dict.get(i.get('PMID')).get('TA').lower())
    elif jt2if_all.get(pubtator_info_dict.get(i.get('PMID')).get('JT').lower()):
        i['IF']=jt2if_all.get(pubtator_info_dict.get(i.get('PMID')).get('JT').lower())
    else:
        i['IF']=0.0
    if jt2if5_brief.get(pubtator_info_dict.get(i.get('PMID')).get('TA').lower()):
        i['IF5']=jt2if5_brief.get(pubtator_info_dict.get(i.get('PMID')).get('TA').lower())
    elif jt2if5_all.get(pubtator_info_dict.get(i.get('PMID')).get('JT').lower()):
        i['IF5']=jt2if5_all.get(pubtator_info_dict.get(i.get('PMID')).get('JT').lower())
    else:
        i['IF5']=0.0

In [86]:
triplelistnew[0]

{'triple': ['SF3B1', 'lead', 'EIF2AK1'],
 'PMID': '35926182',
 'sentence': 'Here, using single-cell technologies and functional validation studies of primary SF3B1-mutant MDS-RS samples, we show that SF3B1 mutations lead to the activation of the EIF2AK1 pathway in response to heme deficiency and that targeting this pathway rescues aberrant erythroid differentiation and enables the red blood cell maturation of MDS-RS erythroblasts.',
 'AB': 'SF3B1 mutations, which occur in 20% of patients with myelodysplastic syndromes (MDS), are the hallmarks of a specific MDS subtype, MDS with ringed sideroblasts (MDS-RS), which is characterized by the accumulation of erythroid precursors in the bone marrow and primarily affects the elderly population. Here, using single-cell technologies and functional validation studies of primary SF3B1-mutant MDS-RS samples, we show that SF3B1 mutations lead to the activation of the EIF2AK1 pathway in response to heme deficiency and that targeting this pathway resc

In [87]:
'''计算同一组triple分别在哪些文章那些句子中出现'''
relation_interpolation_list=[]
relation_interpolation_dict={}
for i in triplelistnew:
    if (i.get('entity_one_transformed'),i.get('triple')[1], i.get('entity_two_transformed')) not in relation_interpolation_dict.keys():
        relation_interpolation_dict[(i.get('entity_one_transformed'),i.get('triple')[1], i.get('entity_two_transformed'))]={'from':entitynum.get(i.get('entity_one_transformed'))-1,
                                   'to':entitynum.get(i.get('entity_two_transformed'))-1,'relation':i.get('triple')[1],
                                   'PMID':[i.get('PMID')],'sentence':[i.get('sentence')],
                                   'DP':[pubtator_info_dict.get(i.get('PMID')).get('DP')],
                                   'TI':[pubtator_info_dict.get(i.get('PMID')).get('TI')],
                                    'TA':[pubtator_info_dict.get(i.get('PMID')).get('TA')],
                                    'IF':[i.get('IF')],   'IF5':[i.get('IF5')],   
                                    'date':[cal_date(pubtator_info_dict.get(i.get('PMID')).get('DP'))],
                                    'first_entity_transformed':i.get('entity_one_transformed'),'second_entity_transformed':i.get('entity_two_transformed'),                                                     
                                    'first_entity':[i.get('entity_one')],'second_entity':[i.get('entity_two')],
                                    'first_type':entitydict_tolist.get(i.get('entity_one_transformed')),'second_type':entitydict_tolist.get(i.get('entity_two_transformed'))}

    else:
        relation_interpolation_dict[(i.get('entity_one_transformed'),i.get('triple')[1], i.get('entity_two_transformed'))]['PMID']=relation_interpolation_dict.get((i.get('entity_one_transformed'),i.get('triple')[1], i.get('entity_two_transformed'))).get('PMID')+[i.get('PMID')]
        relation_interpolation_dict[(i.get('entity_one_transformed'),i.get('triple')[1], i.get('entity_two_transformed'))]['sentence']=relation_interpolation_dict.get((i.get('entity_one_transformed'),i.get('triple')[1], i.get('entity_two_transformed'))).get('sentence')+[i.get('sentence')]
        relation_interpolation_dict[(i.get('entity_one_transformed'),i.get('triple')[1], i.get('entity_two_transformed'))]['first_entity']=relation_interpolation_dict.get((i.get('entity_one_transformed'),i.get('triple')[1], i.get('entity_two_transformed'))).get('first_entity')+[i.get('entity_one')]
        relation_interpolation_dict[(i.get('entity_one_transformed'),i.get('triple')[1], i.get('entity_two_transformed'))]['second_entity']=relation_interpolation_dict.get((i.get('entity_one_transformed'),i.get('triple')[1], i.get('entity_two_transformed'))).get('second_entity')+[i.get('entity_two')]
        relation_interpolation_dict[(i.get('entity_one_transformed'),i.get('triple')[1], i.get('entity_two_transformed'))]['DP']=relation_interpolation_dict.get((i.get('entity_one_transformed'),i.get('triple')[1], i.get('entity_two_transformed'))).get('DP')+[pubtator_info_dict.get(i.get('PMID')).get('DP')]
        relation_interpolation_dict[(i.get('entity_one_transformed'),i.get('triple')[1], i.get('entity_two_transformed'))]['TI']=relation_interpolation_dict.get((i.get('entity_one_transformed'),i.get('triple')[1], i.get('entity_two_transformed'))).get('TI')+[pubtator_info_dict.get(i.get('PMID')).get('TI')]
        relation_interpolation_dict[(i.get('entity_one_transformed'),i.get('triple')[1], i.get('entity_two_transformed'))]['TA']=relation_interpolation_dict.get((i.get('entity_one_transformed'),i.get('triple')[1], i.get('entity_two_transformed'))).get('TA')+[pubtator_info_dict.get(i.get('PMID')).get('TA')]
        relation_interpolation_dict[(i.get('entity_one_transformed'),i.get('triple')[1], i.get('entity_two_transformed'))]['date']=relation_interpolation_dict.get((i.get('entity_one_transformed'),i.get('triple')[1], i.get('entity_two_transformed'))).get('date')+[cal_date(pubtator_info_dict.get(i.get('PMID')).get('DP'))]
        relation_interpolation_dict[(i.get('entity_one_transformed'),i.get('triple')[1], i.get('entity_two_transformed'))]['IF']=relation_interpolation_dict.get((i.get('entity_one_transformed'),i.get('triple')[1], i.get('entity_two_transformed'))).get('IF')+[i.get('IF')]
        relation_interpolation_dict[(i.get('entity_one_transformed'),i.get('triple')[1], i.get('entity_two_transformed'))]['IF5']=relation_interpolation_dict.get((i.get('entity_one_transformed'),i.get('triple')[1], i.get('entity_two_transformed'))).get('IF5')+[i.get('IF5')]

In [88]:
list(relation_interpolation_dict.items())[10]

(('rs1001179', 'investigated', 'Cataract'),
 {'from': 10,
  'to': 5,
  'relation': 'investigated',
  'PMID': ['35556207'],
  'sentence': ['Association of SOD1 50 bp Indel, GPX1 (rs1800668), and CAT (rs1001179) genotypes with all clinical variables was investigated in cataract groups by regression statistics in SPSS  16.0.'],
  'DP': ['2022 Sep'],
  'TI': ['Clinical Association of Biochemical Variations Among Multilocus Genotypes of Antioxidant Enzymes with Susceptibility of Cataract in Hyperglycemia.'],
  'TA': ['Appl Biochem Biotechnol'],
  'IF': [3.094],
  'IF5': [2.973],
  'date': [20220901],
  'first_entity_transformed': 'rs1001179',
  'second_entity_transformed': 'Cataract',
  'first_entity': ['rs1001179'],
  'second_entity': ['cataract'],
  'first_type': ['Mutation'],
  'second_type': ['Disease']})

In [89]:
relation_interpolation_list=list(relation_interpolation_dict.values())

In [90]:
for i in relation_interpolation_list:
    if i.get('from')==0:
        print(i.get('from'))

0
0
0
0
0


In [91]:
# 51551
len(relation_interpolation_list)

100258

In [92]:
'''一个entity对应所有可能的句子和PMID'''
entity2content={}
for i,j in enumerate(triplelistnew):
    if j.get('entity_one_transformed') in entity2content.keys():
        entity2content[j.get('entity_one_transformed')]['sentence']=entity2content.get(j.get('entity_one_transformed')).get('sentence')+[j.get('sentence')]
        entity2content[j.get('entity_one_transformed')]['PMID']=entity2content.get(j.get('entity_one_transformed')).get('PMID')+[j.get('PMID')]
        entity2content[j.get('entity_one_transformed')]['entity']=entity2content.get(j.get('entity_one_transformed')).get('entity')+[j.get('entity_one')]
        entity2content[j.get('entity_one_transformed')]['DP']=entity2content.get(j.get('entity_one_transformed')).get('DP')+[pubtator_info_dict.get(j.get('PMID')).get('DP')]
        entity2content[j.get('entity_one_transformed')]['TI']=entity2content.get(j.get('entity_one_transformed')).get('TI')+[pubtator_info_dict.get(j.get('PMID')).get('TI')]
        entity2content[j.get('entity_one_transformed')]['TA']=entity2content.get(j.get('entity_one_transformed')).get('TA')+[pubtator_info_dict.get(j.get('PMID')).get('TA')]
        entity2content[j.get('entity_one_transformed')]['date']=entity2content.get(j.get('entity_one_transformed')).get('date')+[cal_date(pubtator_info_dict.get(j.get('PMID')).get('DP'))]
        entity2content[j.get('entity_one_transformed')]['IF']=entity2content.get(j.get('entity_one_transformed')).get('IF')+[j.get('IF')]
        entity2content[j.get('entity_one_transformed')]['IF5']=entity2content.get(j.get('entity_one_transformed')).get('IF5')+[j.get('IF5')]
    else:
        entity2content[j.get('entity_one_transformed')]={'entity_transformed':j.get('entity_one_transformed'),'entity':[j.get('entity_one')],'sentence':[j.get('sentence')],'PMID':[j.get('PMID')],
                                                         'DP':[pubtator_info_dict.get(j.get('PMID')).get('DP')],'TI':[pubtator_info_dict.get(j.get('PMID')).get('TI')],
                                                        'TA':[pubtator_info_dict.get(j.get('PMID')).get('TA')],'date':[cal_date(pubtator_info_dict.get(j.get('PMID')).get('DP'))],
                                                        'IF':[j.get('IF')],'IF5':[j.get('IF5')]}
    if j.get('entity_two_transformed') in entity2content.keys():
        entity2content[j.get('entity_two_transformed')]['sentence']=entity2content.get(j.get('entity_two_transformed')).get('sentence')+[j.get('sentence')]
        entity2content[j.get('entity_two_transformed')]['PMID']=entity2content.get(j.get('entity_two_transformed')).get('PMID')+[j.get('PMID')]
        entity2content[j.get('entity_two_transformed')]['entity']=entity2content.get(j.get('entity_two_transformed')).get('entity')+[j.get('entity_two')]
        entity2content[j.get('entity_two_transformed')]['DP']=entity2content.get(j.get('entity_two_transformed')).get('DP')+[pubtator_info_dict.get(j.get('PMID')).get('DP')]
        entity2content[j.get('entity_two_transformed')]['TI']=entity2content.get(j.get('entity_two_transformed')).get('TI')+[pubtator_info_dict.get(j.get('PMID')).get('TI')]
        entity2content[j.get('entity_two_transformed')]['TA']=entity2content.get(j.get('entity_two_transformed')).get('TA')+[pubtator_info_dict.get(j.get('PMID')).get('TA')]
        entity2content[j.get('entity_two_transformed')]['date']=entity2content.get(j.get('entity_two_transformed')).get('date')+[cal_date(pubtator_info_dict.get(j.get('PMID')).get('DP'))]
        entity2content[j.get('entity_two_transformed')]['IF']=entity2content.get(j.get('entity_two_transformed')).get('IF')+[j.get('IF')]
        entity2content[j.get('entity_two_transformed')]['IF5']=entity2content.get(j.get('entity_two_transformed')).get('IF5')+[j.get('IF5')]
    else:
        entity2content[j.get('entity_two_transformed')]={'entity_transformed':j.get('entity_two_transformed'),'entity':[j.get('entity_two')],'sentence':[j.get('sentence')],'PMID':[j.get('PMID')],
                                                         'DP':[pubtator_info_dict.get(j.get('PMID')).get('DP')],'TI':[pubtator_info_dict.get(j.get('PMID')).get('TI')],
                                                        'TA':[pubtator_info_dict.get(j.get('PMID')).get('TA')],'date':[cal_date(pubtator_info_dict.get(j.get('PMID')).get('DP'))],
                                                        'IF':[j.get('IF')],'IF5':[j.get('IF5')]}

In [93]:
# 9480
len(entity2content)

6254

list(entity2content.items())[1041]

In [94]:
entity2list=list(entity2content.values())

entity2list[1001]

In [95]:
len(entity2list)

6254

In [96]:
for i in entity2list:
    i['_id']=i.get('entity_transformed')

In [97]:
entitydict.get(entity2list[0].get('entity_transformed'))

'Gene'

In [98]:
for i in entity2list:
    i['type']=entitydict.get(i.get('entity_transformed')).split(';')

In [99]:
entity2list[10]

{'entity_transformed': 'rs1001179',
 'entity': ['rs1001179'],
 'sentence': ['Association of SOD1 50 bp Indel, GPX1 (rs1800668), and CAT (rs1001179) genotypes with all clinical variables was investigated in cataract groups by regression statistics in SPSS  16.0.'],
 'PMID': ['35556207'],
 'DP': ['2022 Sep'],
 'TI': ['Clinical Association of Biochemical Variations Among Multilocus Genotypes of Antioxidant Enzymes with Susceptibility of Cataract in Hyperglycemia.'],
 'TA': ['Appl Biochem Biotechnol'],
 'date': [20220901],
 'IF': [3.094],
 'IF5': [2.973],
 '_id': 'rs1001179',
 'type': ['Mutation']}

In [100]:
len(entity2list)

6254

In [101]:
import json
json.dump(entity2list,open('results/step4/entityinfo.json','w'))

In [102]:
for i in entity2list:
    if i.get('entity_transformed')=='Lipopolysaccharides':
        print(i.get('type'))

['Carbohydrate', 'Lipid', 'Toxin']


In [103]:
list(relation_interpolation_dict.items())[:3]

[(('SF3B1', 'lead', 'EIF2AK1'),
  {'from': 0,
   'to': 1,
   'relation': 'lead',
   'PMID': ['35926182'],
   'sentence': ['Here, using single-cell technologies and functional validation studies of primary SF3B1-mutant MDS-RS samples, we show that SF3B1 mutations lead to the activation of the EIF2AK1 pathway in response to heme deficiency and that targeting this pathway rescues aberrant erythroid differentiation and enables the red blood cell maturation of MDS-RS erythroblasts.'],
   'DP': ['2022 Nov 2'],
   'TI': ['Targeting the EIF2AK1 Signaling Pathway Rescues Red Blood Cell Production in SF3B1-Mutant Myelodysplastic Syndromes With Ringed Sideroblasts.'],
   'TA': ['Blood Cancer Discov'],
   'IF': [0.0],
   'IF5': [0.0],
   'date': [20221102],
   'first_entity_transformed': 'SF3B1',
   'second_entity_transformed': 'EIF2AK1',
   'first_entity': ['SF3B1'],
   'second_entity': ['EIF2AK1'],
   'first_type': ['Gene'],
   'second_type': ['Gene']}),
 (('SF3B1', 'lead', 'Protoporphyria, Eryt

In [104]:
relation_interpolation_list[10]

{'from': 10,
 'to': 5,
 'relation': 'investigated',
 'PMID': ['35556207'],
 'sentence': ['Association of SOD1 50 bp Indel, GPX1 (rs1800668), and CAT (rs1001179) genotypes with all clinical variables was investigated in cataract groups by regression statistics in SPSS  16.0.'],
 'DP': ['2022 Sep'],
 'TI': ['Clinical Association of Biochemical Variations Among Multilocus Genotypes of Antioxidant Enzymes with Susceptibility of Cataract in Hyperglycemia.'],
 'TA': ['Appl Biochem Biotechnol'],
 'IF': [3.094],
 'IF5': [2.973],
 'date': [20220901],
 'first_entity_transformed': 'rs1001179',
 'second_entity_transformed': 'Cataract',
 'first_entity': ['rs1001179'],
 'second_entity': ['cataract'],
 'first_type': ['Mutation'],
 'second_type': ['Disease']}

In [105]:
len(relation_interpolation_list)

100258

In [106]:
json.dump(relation_interpolation_list,open('results/step4/relationinfo.json','w'))

In [107]:
relation_interpolation_list[100]

{'from': 82,
 'to': 96,
 'relation': 'enforce',
 'PMID': ['35811457'],
 'sentence': ['siRNA-mediated knockdown of SIRT1 and SIRT6 in hASCs from lean adults increased the expression of pro-inflammatory and glycolysis-related markers, and enforced glycogen deposition by overexpression of protein targeting to glycogen (PTG) led to a downregulation of SIRT1/6 protein levels, mimicking the inflammatory state of hASCs from elderly subjects.'],
 'DP': ['2022 Aug'],
 'TI': ['Glycogen accumulation in adipocyte precursors from elderly and obese subjects triggers inflammation via SIRT1/6 signaling.'],
 'TA': ['Aging Cell'],
 'IF': [11.005],
 'IF5': [11.072],
 'date': [20220801],
 'first_entity_transformed': 'SIRT6',
 'second_entity_transformed': 'Glycogen',
 'first_entity': ['SIRT6'],
 'second_entity': ['glycogen'],
 'first_type': ['Gene'],
 'second_type': ['Carbohydrate']}

In [108]:
len(relation_interpolation_list)

100258

In [109]:
len(triplelistnew)

124471

In [110]:
csvf_entity = open("results/step4/entity.csv", "w", newline='', encoding='utf-8')

w_entity = csv.writer(csvf_entity)

In [111]:
w_entity.writerow(("entity:ID", "name","type","pagerank",":LABEL"))

37

In [112]:
for i,j in entitydict.items():
    # if entity2content.get(i):
    w_entity.writerow((entitynum.get(i), i,j,int(entity_countnum.get(i)),j))


In [113]:
csvf_entity.close()

In [114]:
entitycsv=pd.read_csv('results/step4/entity.csv')

In [115]:
entitycsv[:3]

,entity:ID,name,type,pagerank,:LABEL
0,1,SF3B1,Gene,5,Gene
1,2,EIF2AK1,Gene,3,Gene
2,3,"Protoporphyria, Erythropoietic",Disease,6,Disease


In [116]:
len(entitycsv)

6254

In [117]:
csvf_entity = open("results/step4/roles.csv", "w", newline='', encoding='utf-8')

w_entity = csv.writer(csvf_entity)

In [118]:
w_entity.writerow((':START_ID', ':END_ID', "relation", "weight",":TYPE" ))

41

entitynotsame=[]
for i in triplelistnew:
    if i.get('entity_one_transformed')!=i.get('entity_two_transformed'):
        if (i.get('entity_one_transformed'),i.get('triple')[1], i.get('entity_two_transformed')) not in entitynotsame:
            w_entity.writerow((  entitynum.get(i.get('entity_one_transformed')),entitynum.get(i.get('entity_two_transformed')), i.get('triple')[1],int(tripleWeight.get((i.get('entity_one_transformed'),i.get('triple')[1].lower(), i.get('entity_two_transformed')))), i.get('triple')[1]   ))
        else:
            continue
        entitynotsame.append((i.get('entity_one_transformed'),i.get('triple')[1], i.get('entity_two_transformed')))

In [119]:
entitynotsame={}
for i in triplelistnew:
    if i.get('entity_one_transformed')!=i.get('entity_two_transformed'):
        if not entitynotsame.get((i.get('entity_one_transformed'),i.get('triple')[1].lower(), i.get('entity_two_transformed'))) :
            w_entity.writerow((  entitynum.get(i.get('entity_one_transformed')),entitynum.get(i.get('entity_two_transformed')), i.get('triple')[1].lower(),int(tripleWeight.get((i.get('entity_one_transformed'),i.get('triple')[1].lower(), i.get('entity_two_transformed')))), i.get('triple')[1]   ))
        else:
            continue
        entitynotsame[(i.get('entity_one_transformed'),i.get('triple')[1].lower(), i.get('entity_two_transformed'))]=True

In [120]:
entitynum.get('APOE')

867

In [121]:
csvf_entity.close()

In [122]:
import pandas as pd

In [123]:
roles=pd.read_csv("results/step4/roles.csv")

In [124]:
roles.head()

,:START_ID,:END_ID,relation,weight,:TYPE
0,1,2,lead,1,lead
1,1,3,lead,1,lead
2,4,5,indicate,1,indicate
3,5,6,investigated,1,investigated
4,7,5,indicate,1,indicate


In [125]:
relation_list=roles.relation.to_list()

In [126]:
type(relation_list)

list

In [127]:
relation_list[:3]

['lead', 'lead', 'indicate']

In [128]:
#100615
len(relation_list)

100255

In [129]:
len(set(relation_list))

3257

In [130]:
relation_list_set=pd.read_csv('../classofrelations/relation_list_set.csv',header=None,names=['word'])

In [131]:

relation_list_set_dict={}
for i in relation_list_set['word'].to_list():
    relation_list_set_dict[i.lower()]=True

In [132]:
positive_relations=pd.read_csv('../classofrelations/positive-relations.csv',header=None,names=['word'])

In [133]:

positive_relations_dict={}
for i in positive_relations['word'].to_list():
    positive_relations_dict[i.lower()]=True

In [134]:
negative_relations=pd.read_csv('../classofrelations/negative-relations.csv',header=None,names=['word'])

In [135]:

negative_relations_dict={}
for i in negative_relations['word'].to_list():
    negative_relations_dict[i.lower()]=True

In [136]:
negative_relations['word'].to_list()[-5:-1]

['impede', 'postpon', 'be potential targets for', 'impede']

In [137]:
triplelistnew[0]

{'triple': ['SF3B1', 'lead', 'EIF2AK1'],
 'PMID': '35926182',
 'sentence': 'Here, using single-cell technologies and functional validation studies of primary SF3B1-mutant MDS-RS samples, we show that SF3B1 mutations lead to the activation of the EIF2AK1 pathway in response to heme deficiency and that targeting this pathway rescues aberrant erythroid differentiation and enables the red blood cell maturation of MDS-RS erythroblasts.',
 'AB': 'SF3B1 mutations, which occur in 20% of patients with myelodysplastic syndromes (MDS), are the hallmarks of a specific MDS subtype, MDS with ringed sideroblasts (MDS-RS), which is characterized by the accumulation of erythroid precursors in the bone marrow and primarily affects the elderly population. Here, using single-cell technologies and functional validation studies of primary SF3B1-mutant MDS-RS samples, we show that SF3B1 mutations lead to the activation of the EIF2AK1 pathway in response to heme deficiency and that targeting this pathway resc

In [138]:
dropout=[]
for i in triplelistnew:
    if i.get('entity_two_type')=='Disease':
        if i.get('entity_one_type')!='Disease':
            if i.get('entity_one_transformed')!= i.get('entity_two_transformed'):
                if positive_relations_dict.get(i.get('triple')[1].lower()):
                    i['partofspeech']='positive'
                elif negative_relations_dict.get(i.get('triple')[1].lower()):
                    i['partofspeech']='negative'
                else:
                    i['partofspeech']='none'
                    dropout.append(i)

In [139]:
len(dropout)

8389

In [140]:
relation_list=[]
disease_list_raw=[]
for i in triplelistnew:
    if i.get('entity_two_type')=='Disease':
        if i.get('entity_one_type')!='Disease':
            if i.get('entity_one_transformed')!= i.get('entity_two_transformed'):
                relation_list.append(i.get('triple')[1].lower())
                # if i.get('partofspeech')=='positive' or i.get('partofspeech')=='negative':
                disease_list_raw.append(i)

In [141]:
len(relation_list)

17157

In [142]:
#17230
len(disease_list_raw)

17157

In [143]:
new_relation=set()
for i in disease_list_raw:
    if not relation_list_set_dict.get(i.get('triple')[1].lower()):
        new_relation.add(i.get('triple')[1].lower())

In [144]:
len(new_relation)

0

In [145]:
new_relation

set()

In [146]:
len(relation_list_set['word'].to_list())

1576

In [147]:
relation_list_set_new=relation_list_set['word'].to_list()
for i in new_relation:
    relation_list_set_new.append(i)
pd.DataFrame(relation_list_set_new,columns=['word']).to_csv('../classofrelations/relation_list_set.csv')

In [148]:
pd.DataFrame(relation_list_set_new,columns=['word']).to_csv('results/step4/relation_list_set.csv')

# DEAL WITH new_relation!!!!!!!!!!!!

In [149]:
posit=['ascribed','be transcriptional activator via','reveal causal associations with']
negat=['be potential targets for','impede','postpon']
# depre=['promising']

In [150]:
for i in disease_list_raw:
    if i.get('triple')[1]=='reveal causal associations with':
        print(i)
        break

{'triple': ['TNFSF12', 'reveal causal associations with', 'sarcopenia'], 'PMID': '36821647', 'sentence': 'In the female population, AT1B2 and TNFSF12 revealed causal associations with sarcopenia risk according to the EWGSOP criterion; HGF revealed a negative association according to the National Institutes of Health criterion.', 'AB': "BACKGROUND: Identifying sarcopenia's causally associated plasma proteins would provide potential therapeutic targets. METHODS: We screened out sarcopenia-related proteins with genome-wide association studies (GWAS) summary data and cis-protein loci genetic instruments. Summary data of sarcopenia were obtained from a GWAS of 256,523 Europeans aged 60 years and over. The causal effects of the proteins were investigated by cis-Mendelian Randomisation (MR) and multiverse sensitivity analysis. We also explored the robust proteins' causal associations with appendicular lean mass (ALM) and surveyed their druggability and clinical development activities. RESULTS

In [151]:
positive_relations_new=positive_relations['word'].to_list()
for i in posit:
    positive_relations_new.append(i)
pd.DataFrame(positive_relations_new,columns=['word']).to_csv('../classofrelations/positive-relations.csv')

In [152]:
negative_relations_new=negative_relations['word'].to_list()
for i in negat:
    negative_relations_new.append(i)
pd.DataFrame(negative_relations_new,columns=['word']).to_csv('../classofrelations/negative-relations.csv')

In [153]:
pd.DataFrame(positive_relations_new,columns=['word']).to_csv('results/step4/positive-relations.csv')
pd.DataFrame(negative_relations_new,columns=['word']).to_csv('results/step4/negative-relations.csv')

In [154]:
print(len(positive_relations_new))
print(len(negative_relations_new))

308
229


In [155]:
negative_relations['word'].to_list()[-5:-1]

['impede', 'postpon', 'be potential targets for', 'impede']

In [156]:

positive_relations_new_dict={}
for i in positive_relations_new:
    positive_relations_new_dict[i.lower()]=True

In [157]:

negative_relations_new_dict={}
for i in negative_relations_new:
    negative_relations_new_dict[i.lower()]=True

In [158]:
dropout=[]
disease_list=[]
for i in disease_list_raw:
    if positive_relations_new_dict.get(i.get('triple')[1].lower()):
        i['partofspeech']='positive'
        disease_list.append(i)
    elif negative_relations_new_dict.get(i.get('triple')[1].lower()):
        i['partofspeech']='negative'
        disease_list.append(i)
    else:
        i['partofspeech']='none'
        dropout.append(i)

dropout=[]
disease_list=[]
for i in disease_list_raw:
    if i.get('partofspeech')=='positive' or i.get('partofspeech')=='negative':
        disease_list.append(i)

In [159]:
len(disease_list)

8768

In [160]:
import time
import datetime
import re
dropout=[]
months=['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']
import re

for i in disease_list:
    j=pubtator_info_dict.get(i.get('PMID')).get('DP')
    if re.search('[S,s]upplement',j,flags=re.M|re.I):
        j=re.sub('[S,s]upplement.*','',j,flags=re.M|re.I)
    j=re.sub('\s?-\s?','-',j)
    j=re.sub('\.','',j)
    j=re.sub('Aout','Aug',j)
    if len(j.split(' '))==3:
        try:
            date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y %b %d'), '%Y%m%d'))
        except:
            if j.split()[1] in months:
                if re.search(r'\d+',j.split()[2],flags=re.M|re.I):
                    j=j.split()[0]+' '+j.split()[1]+' '+ re.search(r'\d+',j.split()[2],flags=re.M|re.I).group()
                    try:
                        date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y %b %d'), '%Y%m%d'))
                    except:
                        j=j[:4]
                        date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y'), '%Y'+'0101'))
                        print('***************************************ERROR**********************************************')
                else:
                    j=j.split()[0]+' '+j.split()[1]+' '+ '1'
                    try:
                        date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y %b %d'), '%Y%m%d'))
                    except:
                        print(j)
                        j=j[:4]
                        date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y'), '%Y'+'0101'))
            elif j.split()[1] not in months:
                if re.search(r'([a-zA-Z]+)[-/]([a-zA-Z]+)',j.split()[1],flags=re.M|re.I) or re.search(r'([a-zA-Z]+)[-/](\d+)',j.split()[1],flags=re.M|re.I):
                    if re.search(r'([a-zA-Z]+)[-/]([a-zA-Z]+)',j.split()[1],flags=re.M|re.I):
                        middle=re.search(r'([a-zA-Z]+)[-/]([a-zA-Z]+)',j.split()[1],flags=re.M|re.I).group(1)
                    else:
                        middle=re.search(r'([a-zA-Z]+)[-/](\d+)',j.split()[1],flags=re.M|re.I).group(1)
                    if middle in months:
                        if re.search(r'\d+',j.split()[2],flags=re.M|re.I):
                            j=j.split()[0]+' '+middle+' '+ re.search(r'\d+',j.split()[2],flags=re.M|re.I).group()
                        else:
                            j=j.split()[0]+' '+middle+' '+ '1'
                    elif re.match(r'Sum(mer)?',middle):
                        if re.search(r'\d+',j.split()[2],flags=re.M|re.I):
                            j=j.split()[0]+' '+'Jun'+' '+ re.search(r'\d+',j.split()[2],flags=re.M|re.I).group()
                        else:
                            j=j.split()[0]+' '+'Jun'+' '+ '1'
                    elif re.match(r'Spr(ing)?',middle):
                        if re.search(r'\d+',j.split()[2],flags=re.M|re.I):
                            j=j.split()[0]+' '+'Mar'+' '+ re.search(r'\d+',j.split()[2],flags=re.M|re.I).group()
                        else:
                            j=j.split()[0]+' '+'Mar'+' '+ '1'
                    elif re.match(r'Win(ter)?',middle):
                        if re.search(r'\d+',j.split()[2],flags=re.M|re.I):
                            j=j.split()[0]+' '+'Dec'+' '+ re.search(r'\d+',j.split()[2],flags=re.M|re.I).group()
                        else:
                            j=j.split()[0]+' '+'Dec'+' '+ '1'    
                    elif re.match(r'Aut(umn)?',middle) or \
                    re.match(r'Fall',middle):
                        if re.search(r'\d+',j.split()[2],flags=re.M|re.I):
                            j=j.split()[0]+' '+'Sep'+' '+ re.search(r'\d+',j.split()[2],flags=re.M|re.I).group()
                        else:
                            j=j.split()[0]+' '+'Sep'+' '+ '1'      
                    try:
                        date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y %b %d'), '%Y%m%d'))
                    except:
                        print(j)
                        j=j[:4]
                        date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y'), '%Y'+'0101'))
                else:
                    j=j[:4]
                    date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y'), '%Y'+'0101'))
    elif len(j.split())==2:
        if re.match(r'Sum(mer)?',j.split()[1]):
            date=int(j[:4]+'0601')
        elif re.match(r'Spr(ing)?',j.split()[1]):
            date=int(j[:4]+'0301')
        elif re.match(r'Win(ter)?',j.split()[1]):
            date=int(j[:4]+'1201')
        elif re.match(r'Aut(umn)?',j.split()[1]) or re.match(r'Fall',j.split()[1]):
            date=int(j[:4]+'0901')
        elif re.match(r'Sum(mer)?',j.split()[0]):
            date=int(j[-4:]+'0601')
        elif re.match(r'Spr(ing)?',j.split()[0]):
            date=int(j[-4:]+'0301')
        elif re.match(r'Win(ter)?',j.split()[0]):
            date=int(j[-4:]+'1201')
        elif re.match(r'Aut(umn)?',j.split()[0]) or re.match(r'Fall',j.split()[0]):
            date=int(j[-4:]+'0901')
        else:
            try:
                date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y %b'), '%Y%m'+'01'))
            except:
                if re.search(r'([a-zA-Z]+)[-/]([a-zA-Z]+)',j.split()[1],flags=re.M|re.I) or re.search(r'([a-zA-Z]+)[-/](\d+)',j.split()[1],flags=re.M|re.I):
                    if re.search(r'([a-zA-Z]+)[-/]([a-zA-Z]+)',j.split()[1],flags=re.M|re.I):
                        middle=re.search(r'([a-zA-Z]+)[-/]([a-zA-Z]+)',j.split()[1],flags=re.M|re.I).group(1)
                    else:
                        middle=re.search(r'([a-zA-Z]+)[-/](\d+)',j.split()[1],flags=re.M|re.I).group(1)
                    if middle in months:
                        j=j.split()[0]+' '+middle+' '+ '1'
                    elif re.match(r'Sum(mer)?',middle):
                        j=j.split()[0]+' '+'Jun'+' '+ '1'
                    elif re.match(r'Spr(ing)?',middle):
                        j=j.split()[0]+' '+'Mar'+' '+ '1'
                    elif re.match(r'Win(ter)?',middle):
                        j=j.split()[0]+' '+'Dec'+' '+ '1'    
                    elif re.match(r'Aut(umn)?',middle) or \
                    re.match(r'Fall',middle):
                        j=j.split()[0]+' '+'Sep'+' '+ '1'      
                    try:
                        date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y %b %d'), '%Y%m%d'))
                    except:
                        j=j[:4]
                        date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y'), '%Y'+'0101'))
                elif re.search(r'(\d+)[-/](\d+)',j.split()[1],flags=re.M|re.I):
                    midd=re.search(r'(\d+)[-/](\d+)',j.split()[1],flags=re.M|re.I).group(1)
                    if len(midd)!=2:
                        midd='0'+midd
                    j=j[:4]
                    date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y'), '%Y'+midd+'01'))
                else:
                    print(j)
                    j=j[:4]
                    date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y'), '%Y'+'0101'))
    elif len(j.split())==1:
        j=j[:4]
        date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y'), '%Y'+'0101'))
    else:
        j=j.split()[0]+' '+j.split()[1]+' '+re.search(r'\d+',j.split()[2],flags=re.M|re.I).group()
        date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y %b %d'), '%Y%m%d'))
    i['date']=date
    i['TA']=pubtator_info_dict.get(i.get('PMID')).get('TA')
    i['JT']=pubtator_info_dict.get(i.get('PMID')).get('JT')
    i['year']=int(str(date)[:4])
    # del i['_id']
    # i['_id']=i.get('entity_one_type')+'_'+i.get('entity_one_transformed')

In [161]:
disease_list[-1]

{'triple': ['rs11189312', 'related', 'sarcopenia'],
 'PMID': '36573383',
 'sentence': 'Variant rs11189312 was found to be a novel variant affecting ZFYVE27 expressed in skeletal muscles, suggesting that rs11189312 might be related to sarcopenia as a novel discovery of this study.',
 'AB': 'BACKGROUND: Sarcopenia is commonly found in the elderly due to a decline in muscle mass. Many researchers have performed genome-wide association studies (GWAS) to find genetic risk factors of sarcopenia. Although many studies have discovered sarcopenia associated single nucleotide polymorphisms (SNPs), most of them are studies targeting Caucasians. The purpose of this study was to evaluate genetic correlation according to muscle mass in middle aged Koreans using data of the Korean Genome and Epidemiology Study (KOGES), a large population-based genomic cohort study. METHODS: Baseline participants were 10,030 subjects aged 40 to 69 years who were from Ansan or Anseong in Gyeonggi-do, South Korea. Among

In [162]:
for i in disease_list:
    if i.get('triple')[1]=='impede':
        print(i)
        break

{'triple': ['NOTCH1', 'impede', 'carcinogenesis'], 'PMID': '36658434', 'sentence': 'NOTCH1 mutant clones occupy the majority of normal human esophagus by middle age but are comparatively rare in esophageal cancers, suggesting NOTCH1 mutations drive clonal expansion but impede carcinogenesis.', 'AB': 'NOTCH1 mutant clones occupy the majority of normal human esophagus by middle age but are comparatively rare in esophageal cancers, suggesting NOTCH1 mutations drive clonal expansion but impede carcinogenesis. Here we test this hypothesis. Sequencing NOTCH1 mutant clones in aging human esophagus reveals frequent biallelic mutations that block NOTCH1 signaling. In mouse esophagus, heterozygous Notch1 mutation confers a competitive advantage over wild-type cells, an effect enhanced by loss of the second allele. Widespread Notch1 loss alters transcription but has minimal effects on the epithelial structure and cell dynamics. In a carcinogenesis model, Notch1 mutations were less prevalent in tu

In [163]:
# 17233
len(disease_list)

8768

In [164]:
entity_pairs=['Gene',
 'RNA',
'Protein',
 'Carbohydrate',
 'Lipid',
 'Peptide',
 'Pharmaceutical_Preparations',
 
 'Toxin',
 'Mutation',
 'Disease']

names=locals()
for i in entity_pairs:
    names[i+'_typecount']=set()
for i in entity_pairs:
    for j in disease_list:
        if j.get('entity_one_type')==i.replace('_',' '):
            names[i+'_typecount'].add(j.get('entity_one_transformed'))

In [165]:
names=locals()
for i in entity_pairs:
    names[i+'_typecount']=set()
# for i in entity_pairs:
for j in disease_list:
    for i in entitydict_tolist.get(j.get('entity_one_transformed')):
        i=i.replace(' ','_')
        names[i+'_typecount'].add(j.get('entity_one_transformed'))

In [166]:
entitydict_tolist.get('Lipopolysaccharides')

['Carbohydrate', 'Lipid', 'Toxin']

In [167]:
for i in entity_pairs:
    print(i+' '+str(len(list(names[i+'_typecount']))))

Gene 1257
RNA 41
Protein 15
Carbohydrate 77
Lipid 80
Peptide 24
Pharmaceutical_Preparations 2
Toxin 1
Mutation 276
Disease 0


In [168]:
disease_list_copy=copy.deepcopy(disease_list)

In [169]:
disease_list_copy[0]

{'triple': ['SF3B1', 'lead', 'heme deficiency'],
 'PMID': '35926182',
 'sentence': 'Here, using single-cell technologies and functional validation studies of primary SF3B1-mutant MDS-RS samples, we show that SF3B1 mutations lead to the activation of the EIF2AK1 pathway in response to heme deficiency and that targeting this pathway rescues aberrant erythroid differentiation and enables the red blood cell maturation of MDS-RS erythroblasts.',
 'AB': 'SF3B1 mutations, which occur in 20% of patients with myelodysplastic syndromes (MDS), are the hallmarks of a specific MDS subtype, MDS with ringed sideroblasts (MDS-RS), which is characterized by the accumulation of erythroid precursors in the bone marrow and primarily affects the elderly population. Here, using single-cell technologies and functional validation studies of primary SF3B1-mutant MDS-RS samples, we show that SF3B1 mutations lead to the activation of the EIF2AK1 pathway in response to heme deficiency and that targeting this path

In [170]:
for i in disease_list_copy:
    if len(entitydict_tolist.get(i.get('entity_one_transformed')))>1:
        # print(i.get('entity_one_transformed'))
        # print(i.get('entity_one_type'))
        i['entity_one_type']='; '.join(entitydict_tolist.get(i.get('entity_one_transformed')))
        # i['entity_one_type']=entitydict_tolist.get(i.get('entity_one_transformed'))
        # print(i.get('entity_one_type'))
    # else:
    #     i['entity_one_type']=[i.get('entity_one_type')]

In [171]:
json.dump(disease_list_copy,open('results/step4/disease_relation.json','w'))

In [172]:
# Table 4
for j in entity_pairs:
    j=j.replace('_',' ')
    names[j+'_aging']=set()
for i in disease_list:
    if i.get('partofspeech')=='positive':
        for j in entitydict_tolist.get(i.get('entity_one_transformed')):
            j=j.replace('_',' ')
            names[j+'_aging'].add(i.get('entity_one_transformed'))
for j in entity_pairs:    
    j=j.replace('_',' ')
    print(j+'\t'+str(len(names[j+'_aging'])))

Gene	1152
RNA	37
Protein	14
Carbohydrate	64
Lipid	71
Peptide	22
Pharmaceutical Preparations	2
Toxin	1
Mutation	272
Disease	0


In [173]:
for j in entity_pairs:    
    j=j.replace('_',' ')
    print(str(len(names[j+'_aging']))+',', end='')

1152,37,14,64,71,22,2,1,272,0,

In [174]:
# Table 4
for j in entity_pairs:
    j=j.replace('_',' ')
    names[j+'_longevity']=set()
for i in disease_list:
    if i.get('partofspeech')=='negative':
        for j in entitydict_tolist.get(i.get('entity_one_transformed')):
            j=j.replace('_',' ')
            names[j+'_longevity'].add(i.get('entity_one_transformed'))
for j in entity_pairs:    
    j=j.replace('_',' ')
    print(j+'\t'+str(len(names[j+'_longevity'])))

Gene	338
RNA	11
Protein	7
Carbohydrate	50
Lipid	40
Peptide	10
Pharmaceutical Preparations	2
Toxin	1
Mutation	7
Disease	0


In [175]:
Genedict=json.load(open('../dict/Genedict.json'))

In [176]:
list(Genedict.items())[:10]

[('A1BG', ['A1BG']),
 ('A1CF', ['A1CF']),
 ('A2M', ['A2M']),
 ('A2ML1', ['A2ML1']),
 ('A3GALT2', ['A3GALT2']),
 ('A4GALT', ['A4GALT']),
 ('A4GNT', ['A4GNT']),
 ('AAAS', ['AAAS']),
 ('AACS', ['AACS']),
 ('AADAC', ['AADAC'])]

In [177]:
Genedict_upper={}
for i,j in Genedict.items():
    Genedict_upper[i.upper()]=j

In [178]:
num_of_aging=set()
num_of_longevity=set()
for i in disease_list:
    if i.get('partofspeech')=='positive':
        num_of_aging.add(i.get('entity_one_transformed'))
    elif i.get('partofspeech')=='negative':
        num_of_longevity.add(i.get('entity_one_transformed'))
print(len(num_of_aging))
print(len(num_of_longevity))

1622
460


In [179]:
disease_entity_list=[]
disease_entity_triple=set()
disease_entity_dict={}
for i in disease_list:
    if i.get('partofspeech')=='positive':
        if i.get('entity_one_type')=='Gene':
            # if (i.get('entity_one_transformed'),i.get('entity_two_transformed'),i.get('PMID')) not in disease_entity_triple:

            disease_entity_triple.add((i.get('entity_one_transformed'),i.get('entity_two_transformed'),i.get('PMID')))
            disease_entity_list.append(i)
            disease_entity_dict[i.get('entity_one_transformed')]=disease_entity_dict.get(i.get('entity_one_transformed'),0)+1

In [180]:
len(disease_entity_dict)

1152

In [181]:
disease_entity_dict_sorted = sorted(disease_entity_dict.items(), key=lambda x: x[1], reverse=True)

In [182]:
len(disease_entity_dict_sorted)

1152

In [183]:
for i in disease_entity_dict_sorted[:20]:
    print(i)

('INS', 412)
('CRP', 255)
('APOE', 196)
('KL', 180)
('MAPT', 147)
('SIRT1', 102)
('IL6', 92)
('ADIPOQ', 80)
('ALB', 68)
('LEP', 67)
('LMNA', 64)
('WRN', 62)
('CD4', 60)
('TP53', 48)
('CST3', 41)
('SHBG', 41)
('ACE', 39)
('BDNF', 38)
('SIRT6', 36)
('RECQL4', 35)


In [184]:
genage=pd.read_csv('../other_databases/genage_human.csv')

In [185]:
genage[:2]

,GenAge ID,symbol,name,entrez gene id,uniprot,why
0,1,GHR,growth hormone receptor,2690,GHR_HUMAN,mammal
1,2,GHRH,growth hormone releasing hormone,2691,SLIB_HUMAN,mammal


In [186]:
len(genage)

307

In [187]:
genage_symbol=set()
genage_symbol_transfer={}
for i in genage['symbol'].tolist():
    if Genedict_upper.get(i.upper()):
        genage_symbol.add(Genedict_upper.get(i.upper())[0])
        if genage_symbol_transfer.get(Genedict_upper.get(i.upper())[0]) and i not in genage_symbol_transfer.get(Genedict_upper.get(i.upper())[0]):
            genage_symbol_transfer[Genedict_upper.get(i.upper())[0]]+=[i]
            print(Genedict_upper.get(i.upper())[0])
            print(genage_symbol_transfer.get(Genedict_upper.get(i.upper())[0]))
        else:
            genage_symbol_transfer[Genedict_upper.get(i.upper())[0]]=[i]
    else:
        print(i)

TERC


In [188]:
len(genage_symbol)

306

In [189]:
vcx=0
for i in disease_entity_dict_sorted:
    if i[0] in genage_symbol:
        vcx+=1

In [190]:
vcx

160

In [191]:
vcx0=0
for i in disease_entity_dict_sorted[:10]:
    if i[0] in genage_symbol:
        vcx0+=1
print(vcx0)

7


In [192]:
disease_entity_list2=[]
disease_entity_triple2=set()
disease_entity_dict2={}
for i in disease_list:
    if i.get('partofspeech')=='negative':
        if i.get('entity_one_type')=='Gene':
            # if (i.get('entity_one_transformed'),i.get('entity_two_transformed'),i.get('PMID')) not in disease_entity_triple:

            disease_entity_triple2.add((i.get('entity_one_transformed'),i.get('entity_two_transformed'),i.get('PMID')))
            disease_entity_list2.append(i)
            disease_entity_dict2[i.get('entity_one_transformed')]=disease_entity_dict2.get(i.get('entity_one_transformed'),0)+1

In [193]:
disease_entity_dict2_sorted = sorted(disease_entity_dict2.items(), key=lambda x: x[1], reverse=True)

disease_entity_dict3_sorted = sorted(disease_entity_dict3.items(), key=lambda x: x[1], reverse=True)

In [194]:
len(disease_entity_dict2_sorted)

338

In [195]:
for i in disease_entity_dict2_sorted[:20]:
    print(i)

('SIRT1', 61)
('KL', 54)
('INS', 50)
('ACE', 23)
('MAPT', 21)
('SIRT6', 20)
('NFE2L2', 16)
('TP53', 15)
('PRKAA2', 15)
('APOE', 13)
('EPO', 13)
('MTOR', 11)
('SIRT3', 10)
('PTH', 9)
('ADIPOQ', 8)
('WRN', 7)
('PRKAB1', 7)
('CRP', 7)
('AGER', 7)
('SIRT2', 6)


In [196]:
test=set()
for i in disease_entity_dict_sorted[:10]:
    test.add(i[0])
for i in disease_entity_dict2_sorted[:10]:
    if i[0] in test:
        print(i[0])
        
        

SIRT1
KL
INS
MAPT
APOE


In [197]:
test=set()
for i in disease_entity_dict_sorted[:20]:
    test.add(i[0])
for i in disease_entity_dict2_sorted[:20]:
    if i[0] in test:
        print(i[0])
        
        

SIRT1
KL
INS
ACE
MAPT
SIRT6
TP53
APOE
ADIPOQ
WRN
CRP


In [198]:
longevity_human=pd.read_csv('../other_databases/longevity_human.csv')

In [199]:
longevity_human[:2]

,id,Association,Population,Variant(s),Gene(s),PubMed
0,1,non-significant,Dutch,HLA-B40,HLA-B,1859103
1,2,non-significant,Dutch,HLA-DRB5,HLA-DRB5,1859103


In [200]:
len(longevity_human)

550

In [201]:
longevity_human_genes_symbol=set()
for j in longevity_human['Gene(s)'].tolist():
    if type(j)==str:
        for i in j.split(','):
            longevity_human_genes_symbol.add(i)
print(len(longevity_human_genes_symbol))

884


In [202]:

longevity_human_symbol=set()
longevity_human_symbol_transfer={}
for j in longevity_human['Gene(s)'].tolist():
    if type(j)==str:
        for i in j.split(','):
            if Genedict_upper.get(i.upper()):
                longevity_human_symbol.add(Genedict_upper.get(i.upper())[0])
                if longevity_human_symbol_transfer.get(Genedict_upper.get(i.upper())[0]) and i not in longevity_human_symbol_transfer.get(Genedict_upper.get(i.upper())[0]):
                    longevity_human_symbol_transfer[Genedict_upper.get(i.upper())[0]]+=[i]
                    print(Genedict_upper.get(i.upper())[0])
                    print(longevity_human_symbol_transfer.get(Genedict_upper.get(i.upper())[0]))
                else:
                    longevity_human_symbol_transfer[Genedict_upper.get(i.upper())[0]]=[i]

In [203]:
#884
len(longevity_human_symbol)

766

In [204]:
len(disease_entity_dict2_sorted)

338

In [205]:
vcx2=0
for i in disease_entity_dict2_sorted[:10]:
    if i[0] in longevity_human_symbol:
        vcx2+=1

In [206]:
vcx2

7

In [207]:
vcx2=0
for i in disease_entity_dict2_sorted:
    if i[0] in longevity_human_symbol:
        vcx2+=1

In [208]:
#73
vcx2

76

In [209]:
gene_longevity_symbol_dict={}
# gene_longevity_symbol=[]
for i in disease_entity_dict_sorted:
    gene_longevity_symbol_dict[i[0]]=gene_longevity_symbol_dict.get(i[0],0)+i[1]
for i in disease_entity_dict2_sorted:
    gene_longevity_symbol_dict[i[0]]=gene_longevity_symbol_dict.get(i[0],0)+i[1]

In [210]:
len(gene_longevity_symbol_dict)

1257

In [211]:
gene_longevity_symbol = sorted(gene_longevity_symbol_dict.items(), key=lambda x: x[1], reverse=True)

In [212]:
gene_longevity_symbol[:2]

[('INS', 462), ('CRP', 262)]

In [213]:
vcx4=0
for i in gene_longevity_symbol:
    if i[0] in genage_symbol:
        vcx4+=1
print(vcx4)

167


In [214]:
vcx5=0
for i in gene_longevity_symbol:
    if i[0] in longevity_human_symbol:
        vcx5+=1
print(vcx5)

219


In [215]:
vcx44=0
for i in gene_longevity_symbol[:10]:
    if i[0] in genage_symbol:
        vcx44+=1
print(vcx44)

8


In [216]:
vcx55=0
for i in gene_longevity_symbol[:10]:
    if i[0] in longevity_human_symbol:
        vcx55+=1
print(vcx55)

8


In [217]:
print(len(longevity_human_symbol))

766


In [218]:
print(len(disease_entity_dict2_sorted))

338


In [219]:
#Figure 3
for i in disease_entity_dict_sorted[:20]:
    print(i[0])


INS
CRP
APOE
KL
MAPT
SIRT1
IL6
ADIPOQ
ALB
LEP
LMNA
WRN
CD4
TP53
CST3
SHBG
ACE
BDNF
SIRT6
RECQL4


In [220]:
#Figure 3
for i in disease_entity_dict2_sorted[:20]:
    print(i[0])


SIRT1
KL
INS
ACE
MAPT
SIRT6
NFE2L2
TP53
PRKAA2
APOE
EPO
MTOR
SIRT3
PTH
ADIPOQ
WRN
PRKAB1
CRP
AGER
SIRT2


In [221]:
agingbank_genes_pro=pd.read_csv('../other_databases/agingbank_genes_pro.csv')

In [222]:
agingbank_genes_pro[:3]

,Species,Type,name,sample,Tissue,Up/Down,Pro/Anti,Pubmed ID,Details
0,Homo sapiens,gene,ABI3,human,"7 normal thyroid tissues, 21 follicular thyroi...",Down,Pro,21223585,details
1,Homo sapiens,gene,ABI3BP,human,"7 normal thyroid tissues, 21 follicular thyroi...",Down,Pro,21223585,details
2,Homo sapiens,gene,ACE,human,NaN,NaN,Pro,16971231,details


In [223]:
agingbank_genes_pro_list=agingbank_genes_pro['name'].tolist()

In [224]:
len(agingbank_genes_pro_list)

97

In [225]:
# 发现有同一个实体对应不同的名字
agingbank_genes_pro_symbol=set()
agingbank_genes_pro_symbol_transfer={}
for i in agingbank_genes_pro_list:
    if re.search(r'(.*)(\(.*\))',i):
        i=re.search(r'(.*)(\(.*\))',i).groups()[0]
    if re.search(r'(.*)(\s.*)',i):
        i=re.search(r'(.*)(\s.*)',i).groups()[0]
    if Genedict_upper.get(i.upper()):
        agingbank_genes_pro_symbol.add(Genedict_upper.get(i.upper())[0])
        if agingbank_genes_pro_symbol_transfer.get(Genedict_upper.get(i.upper())[0]) and i not in agingbank_genes_pro_symbol_transfer.get(Genedict_upper.get(i.upper())[0]):
            agingbank_genes_pro_symbol_transfer[Genedict_upper.get(i.upper())[0]]+=[i]
            print(Genedict_upper.get(i.upper())[0])
            print(agingbank_genes_pro_symbol_transfer.get(Genedict_upper.get(i.upper())[0]))
        else:
            agingbank_genes_pro_symbol_transfer[Genedict_upper.get(i.upper())[0]]=[i]

IGF1
['IGF1', 'IGF-I']
CDKN2A
['CDKN2A', 'p16']
CDKN2A
['CDKN2A', 'p16', 'p16INK4a']
TP53
['p53', 'TP53']
CDKN2A
['p16INK4a', 'CDKN2A']
TP53
['TP53', 'p53']
CDKN2A
['CDKN2A', 'p16']


In [226]:
len(agingbank_genes_pro_symbol)

54

In [227]:
counts=[]
for i in agingbank_genes_pro_symbol:
    for j in disease_entity_dict_sorted[:10]:
        if i==j[0]:
            counts.append(i)
print(len(counts))
# for i in gene_trans:
#     if i not in counts:
#         print(i)
        

3


In [228]:
counts=[]
for i in agingbank_genes_pro_symbol:
    for j in disease_entity_dict_sorted:
        if i==j[0]:
            counts.append(i)
print(len(counts))
# for i in gene_trans:
#     if i not in counts:
#         print(i)
        

32


In [229]:
32/54

0.5925925925925926

In [230]:
vcx11=0
for i in gene_longevity_symbol:
    if i[0] in agingbank_genes_pro_symbol:
        vcx11+=1
print(vcx11)

33


In [231]:
33/54

0.6111111111111112

In [232]:
vcx66=0
for i in gene_longevity_symbol[:10]:
    if i[0] in agingbank_genes_pro_symbol:
        vcx66+=1
print(vcx66)

3


In [233]:
agingbank_genes_anti=pd.read_csv('../other_databases/agingbank_genes_anti.csv')

In [234]:
agingbank_genes_anti[:3]

,Species,Type,name,sample,Tissue,Up/Down,Pro/Anti,Pubmed ID,Details
0,Homo sapiens,gene,AEG-1,cell line,LNCaP;C42B,Down,Anti,28923415,details
1,Homo sapiens,gene,AKT1,human,Brain,NaN,Anti,16914869,details
2,Homo sapiens,gene,APOE,human,NaN,NaN,Anti,17234815,details


In [235]:
agingbank_genes_anti_list=agingbank_genes_anti['name'].tolist()

In [236]:
len(agingbank_genes_anti_list)

154

In [237]:
# 发现有同一个实体对应不同的名字
agingbank_genes_anti_symbol=set()
agingbank_genes_anti_symbol_transfer={}
for i in agingbank_genes_anti_list:
    if re.search(r'(.*)(\(.*\))',i):
        i=re.search(r'(.*)(\(.*\))',i).groups()[0]
    if re.search(r'(.*)(\s.*)',i):
        i=re.search(r'(.*)(\s.*)',i).groups()[0]
    if Genedict_upper.get(i.upper()):
        agingbank_genes_anti_symbol.add(Genedict_upper.get(i.upper())[0])
        if agingbank_genes_anti_symbol_transfer.get(Genedict_upper.get(i.upper())[0]) and i not in agingbank_genes_anti_symbol_transfer.get(Genedict_upper.get(i.upper())[0]):
            agingbank_genes_anti_symbol_transfer[Genedict_upper.get(i.upper())[0]]+=[i]
            print(Genedict_upper.get(i.upper())[0])
            print(agingbank_genes_anti_symbol_transfer.get(Genedict_upper.get(i.upper())[0]))
        else:
            agingbank_genes_anti_symbol_transfer[Genedict_upper.get(i.upper())[0]]=[i]

FOXO3
['FOXO3', 'FOXO3a']
PARP1
['PARP', 'PARP1']
STAG2
['SA2', 'STAG2']
TERT
['hEST2', 'TERT']
KL
['klotho', 'KL']


In [238]:
len(agingbank_genes_anti_symbol)

96

In [239]:
counts2=[]
for i in agingbank_genes_anti_symbol:
    for j in disease_entity_dict2_sorted[:10]:
        if i==j[0]:
            counts2.append(i)
print(len(counts2))
# for i in gene_trans:
#     if i not in counts:
#         print(i)
        

6


In [240]:
counts2=[]
for i in agingbank_genes_anti_symbol:
    for j in disease_entity_dict2_sorted:
        if i==j[0]:
            counts2.append(i)
print(len(counts2))
# for i in gene_trans:
#     if i not in counts:
#         print(i)
        

28


In [241]:
28/96

0.2916666666666667

In [242]:
vcx12=0
for i in gene_longevity_symbol:
    if i[0] in agingbank_genes_anti_symbol:
        vcx12+=1
print(vcx12)

46


In [243]:
46/96

0.4791666666666667

In [244]:
vcx77=0
for i in gene_longevity_symbol[:10]:
    if i[0] in agingbank_genes_anti_symbol:
        vcx77+=1
print(vcx77)

6


In [245]:
agingatlas_genes=pd.read_csv('../other_databases/agingatlas_genes.csv')

In [246]:
agingatlas_genes[:3]

,Symbol
0,IL2RB
1,PSAT1
2,A2M


In [247]:
agingatlas_genes_list=agingatlas_genes['Symbol'].tolist()

In [248]:
len(agingatlas_genes_list)

502

In [249]:
# 发现有同一个实体对应不同的名字
agingatlas_genes_symbol=set()
agingatlas_genes_symbol_transfer={}
for i in agingatlas_genes_list:
    if Genedict_upper.get(i.upper()):
        if Genedict_upper.get(i.upper())[0] in agingatlas_genes_symbol:
            print(Genedict_upper.get(i.upper())[0])
            print(i)
        agingatlas_genes_symbol.add(Genedict_upper.get(i.upper())[0])

        if agingatlas_genes_symbol_transfer.get(Genedict_upper.get(i.upper())[0]) and i not in agingatlas_genes_symbol_transfer.get(Genedict_upper.get(i.upper())[0]):
            agingatlas_genes_symbol_transfer[Genedict_upper.get(i.upper())[0]]+=[i]
            print(Genedict_upper.get(i.upper())[0])
            print(agingatlas_genes_symbol_transfer.get(Genedict_upper.get(i.upper())[0]))
        else:
            agingatlas_genes_symbol_transfer[Genedict_upper.get(i.upper())[0]]=[i]
    else:
        print(i)

RAD27
TERC


In [250]:
len(agingatlas_genes_symbol)

500

In [251]:
counts=[]
for i in agingatlas_genes_symbol:
    for j in disease_entity_dict_sorted[:10]:
        if i==j[0]:
            counts.append(i)
print(len(counts))
# for i in gene_trans:
#     if i not in counts:
#         print(i)

8


In [252]:
counts=[]
for i in agingatlas_genes_symbol:
    for j in disease_entity_dict_sorted:
        if i==j[0]:
            counts.append(i)
print(len(counts))
# for i in gene_trans:
#     if i not in counts:
#         print(i)
        

228


In [253]:
228/500

0.456

In [254]:
vcx88=0
for i in gene_longevity_symbol[:10]:
    if i[0] in agingatlas_genes_symbol:
        vcx88+=1
print(vcx88)

9


In [255]:
disease_entity_dict_sorted[:2]

[('INS', 412), ('CRP', 255)]

In [256]:
gene_longevity_symbol[:2]

[('INS', 462), ('CRP', 262)]

In [257]:
vcx13=0
for i in gene_longevity_symbol:
    if i[0] in agingatlas_genes_symbol:
        vcx13+=1
print(vcx13)

238


In [258]:
238/500

0.476

In [259]:
#Supplementary Table 1

datas1= []
datas2=[]
datas3=[]
datas4=[]
datas5=[]
for i in disease_entity_dict_sorted:
    datas1.append(i[0])
    datas2.append(i[1])
    if genage_symbol_transfer.get(i[0]):
        datas3.append('; '.join(genage_symbol_transfer.get(i[0])))
    else:
        datas3.append('')
    if agingbank_genes_pro_symbol_transfer.get(i[0]):
        datas4.append('; '.join(agingbank_genes_pro_symbol_transfer.get(i[0])))
    else:
        datas4.append('') 
    if agingatlas_genes_symbol_transfer.get(i[0]):
        datas5.append('; '.join(agingatlas_genes_symbol_transfer.get(i[0])))
    else:
        datas5.append('') 
dataframe1 = pd.DataFrame({'HALD':datas1,'Frequency of occurrence':datas2,'GenAge':datas3,'Aging Atlas':datas5,'AgingBank (Pro)':datas4})
dataframe1.to_csv("results/step4/Supplementary1.csv", index=False)

In [260]:
#Supplementary Table 2

datas6= []
datas7=[]
datas8=[]
datas9=[]
datas10=[]
for i in disease_entity_dict2_sorted:
    datas6.append(i[0])
    datas7.append(i[1])
    if longevity_human_symbol_transfer.get(i[0]):
        datas8.append('; '.join(longevity_human_symbol_transfer.get(i[0])))
    else:
        datas8.append('')
    if agingbank_genes_anti_symbol_transfer.get(i[0]):
        datas9.append('; '.join(agingbank_genes_anti_symbol_transfer.get(i[0])))
    else:
        datas9.append('') 
dataframe2 = pd.DataFrame({'HALD':datas6,'Frequency of occurrence':datas7,'LongevityMap':datas8,'AgingBank (Anti)':datas9})
dataframe2.to_csv("results/step4/Supplementary2.csv", index=False)

In [261]:
#Supplementary Table 3

data1= []
data2=[]
data3=[]
data4=[]
data5=[]
data6=[]
data7=[]
data8=[]
for i in gene_longevity_symbol:
    label1=[]
    label2=[]
    data1.append(i[0])
    for j in disease_entity_dict_sorted:
        if i[0]==j[0]:
            label1.append('Aging')
            label2.append(str(j[1]))
            break
    for j in disease_entity_dict2_sorted:
        if i[0]==j[0]:
            label1.append('Longevity')
            label2.append(str(j[1]))
            break
    data2.append(' | '.join(label1))
    data3.append(' | '.join(label2))
    if genage_symbol_transfer.get(i[0]):
        data4.append('; '.join(genage_symbol_transfer.get(i[0])))
    else:
        data4.append('')
    if longevity_human_symbol_transfer.get(i[0]):
        data5.append('; '.join(longevity_human_symbol_transfer.get(i[0])))
    else:
        data5.append('')
    if agingbank_genes_pro_symbol_transfer.get(i[0]):
        data6.append('; '.join(agingbank_genes_pro_symbol_transfer.get(i[0])))
    else:
        data6.append('') 
    if agingbank_genes_anti_symbol_transfer.get(i[0]):
        data7.append('; '.join(agingbank_genes_anti_symbol_transfer.get(i[0])))
    else:
        data7.append('') 
    if agingatlas_genes_symbol_transfer.get(i[0]):
        data8.append('; '.join(agingatlas_genes_symbol_transfer.get(i[0])))
    else:
        data8.append('') 
dataframe3 = pd.DataFrame({'HALD':data1,'Aging or Longevity':data2,'Frequency of occurrence':data3,'GenAge':data4,'LongevityMap':data5,'Aging Atlas':data8,'AgingBank (Pro)':data6,'AgingBank (Anti)':data7})
dataframe3.to_csv("results/step4/Supplementary3.csv", index=False)

In [262]:
'''
disease_entity_dict_sorted
disease_entity_dict2_sorted
genage_symbol
longevity_human_symbol
agingbank_genes_pro_symbol
agingbank_genes_anti_symbol
agingatlas_genes_symbol
'''


'\ndisease_entity_dict_sorted\ndisease_entity_dict2_sorted\ngenage_symbol\nlongevity_human_symbol\nagingbank_genes_pro_symbol\nagingbank_genes_anti_symbol\nagingatlas_genes_symbol\n'

In [263]:
ccc=0
for i in genage_symbol:
    if i in agingatlas_genes_symbol:
        ccc+=1
print(ccc)
ccc=0
for i in genage_symbol:
    if i in agingbank_genes_pro_symbol:
        ccc+=1
print(ccc)
ccc=0
for i in agingatlas_genes_symbol:
    if i in agingbank_genes_pro_symbol:
        ccc+=1
print(ccc)
ccc=0
for i in disease_entity_dict_sorted:
    if i[0] in genage_symbol and i[0] in agingatlas_genes_symbol:
        ccc+=1
print(ccc)
ccc=0
for i in disease_entity_dict_sorted:
    if i[0] in genage_symbol and i[0] in agingbank_genes_pro_symbol:
        ccc+=1
print(ccc)
ccc=0
for i in disease_entity_dict_sorted:
    if i[0] in agingatlas_genes_symbol and i[0] in agingbank_genes_pro_symbol:
        ccc+=1
print(ccc)
ccc=0
for i in genage_symbol:
    if i in agingatlas_genes_symbol and i in agingbank_genes_pro_symbol:
        ccc+=1
print(ccc)
ccc=0
for i in disease_entity_dict_sorted:
    if i[0] in genage_symbol and i[0] in agingatlas_genes_symbol and i[0] in agingbank_genes_pro_symbol :
        ccc+=1
print(ccc)

306
16
20
160
14
17
16
14


In [264]:
ccc=0
for i in longevity_human_symbol:
    if i in agingbank_genes_anti_symbol:
        ccc+=1
print(ccc)
ccc=0
for i in disease_entity_dict2_sorted:
    if i[0] in longevity_human_symbol and i[0] in agingbank_genes_anti_symbol:
        ccc+=1
print(ccc)

33
22


In [265]:
relation_interpolation_list[0]

{'from': 0,
 'to': 1,
 'relation': 'lead',
 'PMID': ['35926182'],
 'sentence': ['Here, using single-cell technologies and functional validation studies of primary SF3B1-mutant MDS-RS samples, we show that SF3B1 mutations lead to the activation of the EIF2AK1 pathway in response to heme deficiency and that targeting this pathway rescues aberrant erythroid differentiation and enables the red blood cell maturation of MDS-RS erythroblasts.'],
 'DP': ['2022 Nov 2'],
 'TI': ['Targeting the EIF2AK1 Signaling Pathway Rescues Red Blood Cell Production in SF3B1-Mutant Myelodysplastic Syndromes With Ringed Sideroblasts.'],
 'TA': ['Blood Cancer Discov'],
 'IF': [0.0],
 'IF5': [0.0],
 'date': [20221102],
 'first_entity_transformed': 'SF3B1',
 'second_entity_transformed': 'EIF2AK1',
 'first_entity': ['SF3B1'],
 'second_entity': ['EIF2AK1'],
 'first_type': ['Gene'],
 'second_type': ['Gene']}

In [266]:
triplelistnew[0]

{'triple': ['SF3B1', 'lead', 'EIF2AK1'],
 'PMID': '35926182',
 'sentence': 'Here, using single-cell technologies and functional validation studies of primary SF3B1-mutant MDS-RS samples, we show that SF3B1 mutations lead to the activation of the EIF2AK1 pathway in response to heme deficiency and that targeting this pathway rescues aberrant erythroid differentiation and enables the red blood cell maturation of MDS-RS erythroblasts.',
 'AB': 'SF3B1 mutations, which occur in 20% of patients with myelodysplastic syndromes (MDS), are the hallmarks of a specific MDS subtype, MDS with ringed sideroblasts (MDS-RS), which is characterized by the accumulation of erythroid precursors in the bone marrow and primarily affects the elderly population. Here, using single-cell technologies and functional validation studies of primary SF3B1-mutant MDS-RS samples, we show that SF3B1 mutations lead to the activation of the EIF2AK1 pathway in response to heme deficiency and that targeting this pathway resc

In [267]:
direction=set()
direction_list=[]
direction_type=[]
for i in triplelistnew:
    for j in entitydict_tolist.get(i.get('entity_one_transformed')):
        for k in entitydict_tolist.get(i.get('entity_two_transformed')):
            if (i.get('entity_one_transformed'),(i.get('entity_two_transformed')),j,k) not in direction:
                direction.add((i.get('entity_one_transformed'),(i.get('entity_two_transformed')),j,k))
                direction_list.append((i.get('entity_one_transformed'),(i.get('entity_two_transformed'))))
                direction_type.append((j.replace(' ','_'),k.replace(' ','_')))

In [268]:
#51622
len(direction_list)

52272

In [269]:
type_dict={}
type_dict_ensure={}
for i in range(len(direction_list)):
    if not type_dict_ensure.get((direction_list[i][0],direction_list[i][1],direction_type[i][0],direction_type[i][1])):
        type_dict[(direction_list[i][0],direction_list[i][1])]=tuple((direction_type[i][0],direction_type[i][1]))
        type_dict_ensure[(direction_list[i][0],direction_list[i][1],direction_type[i][0],direction_type[i][1])]=tuple((direction_type[i][0],direction_type[i][1]))

In [270]:
#51203
len(type_dict)

51841

In [271]:
entity_pairs

['Gene',
 'RNA',
 'Protein',
 'Carbohydrate',
 'Lipid',
 'Peptide',
 'Pharmaceutical_Preparations',
 'Toxin',
 'Mutation',
 'Disease']

In [272]:
entity_pairs=['Gene',
 'RNA',
'Protein',
 'Carbohydrate',
 'Lipid',
 'Peptide',
 'Pharmaceutical_Preparations',
 
 'Toxin',
 'Mutation',
 'Disease']

In [273]:
entity_pairs_dict={}
entity_pairs_dict_num={}
for i in entity_pairs:
    temp={}
    temp2={}
    for j in entity_pairs:
        temp.update({j:0})
        temp2.update({j:set()})
    entity_pairs_dict_num[i]=temp
    entity_pairs_dict[i]=temp2

In [274]:
entity_pairs_dict

{'Gene': {'Gene': set(),
  'RNA': set(),
  'Protein': set(),
  'Carbohydrate': set(),
  'Lipid': set(),
  'Peptide': set(),
  'Pharmaceutical_Preparations': set(),
  'Toxin': set(),
  'Mutation': set(),
  'Disease': set()},
 'RNA': {'Gene': set(),
  'RNA': set(),
  'Protein': set(),
  'Carbohydrate': set(),
  'Lipid': set(),
  'Peptide': set(),
  'Pharmaceutical_Preparations': set(),
  'Toxin': set(),
  'Mutation': set(),
  'Disease': set()},
 'Protein': {'Gene': set(),
  'RNA': set(),
  'Protein': set(),
  'Carbohydrate': set(),
  'Lipid': set(),
  'Peptide': set(),
  'Pharmaceutical_Preparations': set(),
  'Toxin': set(),
  'Mutation': set(),
  'Disease': set()},
 'Carbohydrate': {'Gene': set(),
  'RNA': set(),
  'Protein': set(),
  'Carbohydrate': set(),
  'Lipid': set(),
  'Peptide': set(),
  'Pharmaceutical_Preparations': set(),
  'Toxin': set(),
  'Mutation': set(),
  'Disease': set()},
 'Lipid': {'Gene': set(),
  'RNA': set(),
  'Protein': set(),
  'Carbohydrate': set(),
  'Lipi

In [275]:
for i in range(len(direction_list)):
        if direction_list[i][0] not in entity_pairs_dict.get(direction_type[i][1]).get(direction_type[i][0]):
            entity_pairs_dict[direction_type[i][1]][direction_type[i][0]].add(direction_list[i][0])
            entity_pairs_dict_num[direction_type[i][1]][direction_type[i][0]]+=1
            

In [276]:
entity_pairs_dict_num

{'Gene': {'Gene': 1062,
  'RNA': 34,
  'Protein': 9,
  'Carbohydrate': 50,
  'Lipid': 45,
  'Peptide': 19,
  'Pharmaceutical_Preparations': 0,
  'Toxin': 1,
  'Mutation': 236,
  'Disease': 689},
 'RNA': {'Gene': 38,
  'RNA': 6,
  'Protein': 0,
  'Carbohydrate': 1,
  'Lipid': 0,
  'Peptide': 0,
  'Pharmaceutical_Preparations': 0,
  'Toxin': 0,
  'Mutation': 1,
  'Disease': 35},
 'Protein': {'Gene': 31,
  'RNA': 0,
  'Protein': 3,
  'Carbohydrate': 7,
  'Lipid': 3,
  'Peptide': 0,
  'Pharmaceutical_Preparations': 0,
  'Toxin': 0,
  'Mutation': 0,
  'Disease': 78},
 'Carbohydrate': {'Gene': 149,
  'RNA': 3,
  'Protein': 3,
  'Carbohydrate': 64,
  'Lipid': 34,
  'Peptide': 7,
  'Pharmaceutical_Preparations': 2,
  'Toxin': 0,
  'Mutation': 6,
  'Disease': 287},
 'Lipid': {'Gene': 164,
  'RNA': 2,
  'Protein': 2,
  'Carbohydrate': 27,
  'Lipid': 68,
  'Peptide': 4,
  'Pharmaceutical_Preparations': 0,
  'Toxin': 1,
  'Mutation': 15,
  'Disease': 310},
 'Peptide': {'Gene': 49,
  'RNA': 0,
  'P

In [277]:
#8581
countnum=0
for i in entity_pairs_dict_num.values():
    for j in i.values():
        countnum+=j
print(countnum)

8683


In [278]:
relation_display=[]
for i,j in entity_pairs_dict_num.items():
    for k,l in j.items():
        relation_display.append([k,i+' ',l])

In [279]:
j

{'Gene': 1718,
 'RNA': 57,
 'Protein': 18,
 'Carbohydrate': 103,
 'Lipid': 106,
 'Peptide': 32,
 'Pharmaceutical_Preparations': 7,
 'Toxin': 2,
 'Mutation': 368,
 'Disease': 2166}

In [280]:
relation_display

[['Gene', 'Gene ', 1062],
 ['RNA', 'Gene ', 34],
 ['Protein', 'Gene ', 9],
 ['Carbohydrate', 'Gene ', 50],
 ['Lipid', 'Gene ', 45],
 ['Peptide', 'Gene ', 19],
 ['Pharmaceutical_Preparations', 'Gene ', 0],
 ['Toxin', 'Gene ', 1],
 ['Mutation', 'Gene ', 236],
 ['Disease', 'Gene ', 689],
 ['Gene', 'RNA ', 38],
 ['RNA', 'RNA ', 6],
 ['Protein', 'RNA ', 0],
 ['Carbohydrate', 'RNA ', 1],
 ['Lipid', 'RNA ', 0],
 ['Peptide', 'RNA ', 0],
 ['Pharmaceutical_Preparations', 'RNA ', 0],
 ['Toxin', 'RNA ', 0],
 ['Mutation', 'RNA ', 1],
 ['Disease', 'RNA ', 35],
 ['Gene', 'Protein ', 31],
 ['RNA', 'Protein ', 0],
 ['Protein', 'Protein ', 3],
 ['Carbohydrate', 'Protein ', 7],
 ['Lipid', 'Protein ', 3],
 ['Peptide', 'Protein ', 0],
 ['Pharmaceutical_Preparations', 'Protein ', 0],
 ['Toxin', 'Protein ', 0],
 ['Mutation', 'Protein ', 0],
 ['Disease', 'Protein ', 78],
 ['Gene', 'Carbohydrate ', 149],
 ['RNA', 'Carbohydrate ', 3],
 ['Protein', 'Carbohydrate ', 3],
 ['Carbohydrate', 'Carbohydrate ', 64],
 ['

In [281]:
entity_pairs

['Gene',
 'RNA',
 'Protein',
 'Carbohydrate',
 'Lipid',
 'Peptide',
 'Pharmaceutical_Preparations',
 'Toxin',
 'Mutation',
 'Disease']

In [282]:
for i in entity_pairs:
    xs=[]
    for j in relation_display:
        if j[0]==i.replace(' ','_'):
            xs.append(str(j[-1]))
    print('\t'.join(xs))

1062	38	31	149	164	49	0	20	133	1718
34	6	0	3	2	0	0	2	1	57
9	0	3	3	2	1	0	0	0	18
50	1	7	64	27	7	0	0	2	103
45	0	3	34	68	9	0	3	8	106
19	0	0	7	4	7	0	0	1	32
0	0	0	2	0	1	0	0	0	7
1	0	0	0	1	0	0	2	0	2
236	1	0	6	15	1	0	0	100	368
689	35	78	287	310	91	13	18	141	2166


In [283]:
all_es=json.load(open('results/step2/all_es_lack.json'))

In [284]:
entity2aging={}
entity2longevity={}
for i in triplelistnew:
    if i.get('entity_two_type')=='Disease':
        if i.get('entity_one_type')!='Disease':
            if i.get('entity_one_transformed')!= i.get('entity_two_transformed'):
                if positive_relations_dict.get(i.get('triple')[1].lower()):
                    entity2aging[i.get('entity_one_transformed')]=True
                elif negative_relations_dict.get(i.get('triple')[1].lower()):
                    entity2longevity[i.get('entity_one_transformed')]=True

In [285]:
len(entity2aging)

1622

In [286]:
len(entity2longevity)

460

In [287]:
aaaa=0
for i in entity2aging.keys():
    if entity2longevity.get(i):
        aaaa+=1
print(aaaa)

323


In [288]:
for i in all_es:
    if entitydict.get(i.get('entity')):
        i['network']=True
    else:
        i['network']=False
    if entity2aging.get(i.get('entity')):
        i['aging']=True
    else:
        i['aging']=False
    if entity2longevity.get(i.get('entity')):
        i['longevity']=True
    else:
        i['longevity']=False

In [289]:
json.dump(all_es,open('results/step4/all_es.json','w'))

In [290]:
# 10907
len(all_es)

11068

In [290]:
HALD_genes_transfer={}
aging_genes_set=set()
for i in disease_entity_dict_sorted:
    HALD_genes_transfer[i[0]]=i[0]
    aging_genes_set.add(i[0])

In [291]:
len(list(aging_genes_set))

1152

In [292]:
print(len(genage_symbol_transfer))
for i in genage_symbol_transfer.keys():
    aging_genes_set.add(i)
print(len(aging_genes_set))
print(len(agingbank_genes_pro_symbol_transfer))
for i in agingbank_genes_pro_symbol_transfer.keys():
    aging_genes_set.add(i)
print(len(aging_genes_set))
print(len(agingatlas_genes_symbol_transfer))
for i in agingatlas_genes_symbol_transfer.keys():
    aging_genes_set.add(i)
print(len(aging_genes_set))

306
1298
54
1318
500
1443


In [293]:
aging_genes_list=list(aging_genes_set)

In [294]:
gff=0
for i in HALD_01:
    if i==1:
        gff+=1
gff

0

In [295]:
HALD_01=[]
genage_symbol_01=[]
agingbank_genes_pro_symbol_01=[]
agingatlas_genes_symbol_01=[]
for i in aging_genes_list:
    if HALD_genes_transfer.get(i):
        HALD_01.append(1)
    else:
        HALD_01.append(0)
    if genage_symbol_transfer.get(i):
        genage_symbol_01.append(1)
    else:
        genage_symbol_01.append(0)
    if agingbank_genes_pro_symbol_transfer.get(i):
        agingbank_genes_pro_symbol_01.append(1)
    else:
        agingbank_genes_pro_symbol_01.append(0)
    if agingatlas_genes_symbol_transfer.get(i):
        agingatlas_genes_symbol_01.append(1)
    else:
        agingatlas_genes_symbol_01.append(0)

In [296]:
vfdsvg=0
for i in HALD_genes_transfer.keys():
    if not genage_symbol_transfer.get(i) and  agingbank_genes_pro_symbol_transfer.get(i) and  agingatlas_genes_symbol_transfer.get(i):
        vfdsvg+=1
vfdsvg

3

In [297]:
dataframe5 = pd.DataFrame({'HALD':HALD_01,'GenAge':genage_symbol_01,'Aging Atlas':agingatlas_genes_symbol_01,'AgingBank (Pro)':agingbank_genes_pro_symbol_01})
dataframe5.to_csv("results/step4/upset1.csv", index=False,sep=',')

In [298]:
HALD_longevity_genes_transfer={}
longevity_genes_set=set()
for i in disease_entity_dict2_sorted:
    HALD_longevity_genes_transfer[i[0]]=i[0]
    longevity_genes_set.add(i[0])

In [299]:
len(list(longevity_genes_set))

338

In [300]:
print(len(longevity_human_symbol_transfer))
for i in longevity_human_symbol_transfer.keys():
    longevity_genes_set.add(i)
print(len(longevity_genes_set))
print(len(agingbank_genes_anti_symbol_transfer))
for i in agingbank_genes_anti_symbol_transfer.keys():
    longevity_genes_set.add(i)
print(len(longevity_genes_set))


766
1028
96
1085


In [301]:
longevity_genes_list=list(longevity_genes_set)

In [302]:
HALD_longevity_01=[]
longevity_human_symbol_01=[]
agingbank_genes_anti_symbol_01=[]

for i in longevity_genes_list:
    if HALD_longevity_genes_transfer.get(i):
        HALD_longevity_01.append(1)
    else:
        HALD_longevity_01.append(0)
    if longevity_human_symbol_transfer.get(i):
        longevity_human_symbol_01.append(1)
    else:
        longevity_human_symbol_01.append(0)
    if agingbank_genes_anti_symbol_transfer.get(i):
        agingbank_genes_anti_symbol_01.append(1)
    else:
        agingbank_genes_anti_symbol_01.append(0)

In [303]:
dataframe6 = pd.DataFrame({'HALD':HALD_longevity_01,'LongevityMap':longevity_human_symbol_01,'AgingBank (Anti)':agingbank_genes_anti_symbol_01})
dataframe6.to_csv("results/step4/upset2.csv", index=False,sep=',')

In [304]:
pwd

'/root/hald_autoupdate/2023_03_13'

In [305]:
entity_pairs

['Gene',
 'RNA',
 'Protein',
 'Carbohydrate',
 'Lipid',
 'Peptide',
 'Pharmaceutical_Preparations',
 'Toxin',
 'Mutation',
 'Disease']

In [306]:
entity_pairs=['Gene',
 'RNA',
'Protein',
 'Carbohydrate',
 'Lipid',
 'Peptide',
 'Pharmaceutical_Preparations',

 'Toxin',
 'Mutation',
 'Disease']

In [321]:
names=locals()
for i in entity_pairs:
    names[i+'_all']=[]
    for j in all_es:
        if j.get('type')==i.replace('_',' '):
            names[i+'_all'].append(j)

In [322]:
len(Gene_all)

4953

In [323]:
len(Disease_all)

3347

In [324]:
Gene_all_sorted=sorted(Gene_all,key=lambda x:x['literature'],reverse=True)

In [325]:
0.21+0.18+0.15+0.13+0.10+0.06+0.05+0.05+0.04+0.04

1.0100000000000002

In [326]:
for i in entity_pairs:
    print('\n')
    print(i)
    names[i+'_all_sorted']=sorted(names[i+'_all'],key=lambda x:x['literature'],reverse=True)
    names[i+'_all_sorted_num']=0
    for j in names[i+'_all_sorted'][:10]:
        print("'"+j.get('entity')+"',")
for i in entity_pairs:
    print('\n')
    print(i)

    for j in names[i+'_all_sorted'][:10]:
        print(str(j.get('literature'))+",")
        names[i+'_all_sorted_num']+=j.get('literature')
for i in entity_pairs:
    print('\n')
    print(i)
    cxv=0
    for j in names[i+'_all_sorted'][:10]:
        z=j.get('literature')*100/names[i+'_all_sorted_num']
        print(str(round(z,2))+",")
    #     cxv+=round(z,2)
    # print(cxv)



Gene
'INS',
'CRP',
'APOE',
'IL6',
'CD4',
'TNF',
'MAPT',
'ALB',
'SIRT1',
'CD8A',


RNA
'H19',
'MIR34A',
'TERC',
'MIR21',
'MIR146A',
'MIR155',
'MIR221',
'MIR132',
'MIR29A',
'HCCAT5',


Protein
'Infliximab',
'Rituximab',
'Bevacizumab',
'Denosumab',
'Trastuzumab',
'Palivizumab',
'Cetuximab',
'Nivolumab',
'Adalimumab',
'Ranibizumab',


Carbohydrate
'Glucose',
'Ascorbic Acid',
'Hyaluronic Acid',
'Sugars',
'Polysaccharides',
'Heparin',
'Digoxin',
'Lipopolysaccharides',
'Doxorubicin',
'Fluorodeoxyglucose F18',


Lipid
'Cholesterol',
'Triglycerides',
'Fatty Acids',
'Fatty Acids, Unsaturated',
'Fatty Acids, Omega-3',
'Lipofuscin',
'Lipopolysaccharides',
'Phospholipids',
'Cholecalciferol',
'Docosahexaenoic Acids',


Peptide
'Glutathione',
'Cyclosporine',
'Vancomycin',
'Thyrotropin',
'Ghrelin',
'Bleomycin',
'Enalapril',
'Teriparatide',
'Glutathione Disulfide',
'Dipeptides',


Pharmaceutical_Preparations
'Trimethoprim, Sulfamethoxazole Drug Combination',
'Amoxicillin-Potassium Clavulanate Combina

In [327]:
for i in entity_pairs:
    print('\n')
    print(i)
    print(names[i+'_all_sorted_num'])




Gene
10941


RNA
299


Protein
703


Carbohydrate
5857


Lipid
6656


Peptide
1295


Pharmaceutical_Preparations
167


Toxin
266


Mutation
221


Disease
121621


In [328]:
for i in entity_pairs:
    print(i)
    print(len(names[i+'_all']))


Gene
4953
RNA
344
Protein
29
Carbohydrate
203
Lipid
191
Peptide
79
Pharmaceutical_Preparations
11
Toxin
18
Mutation
1893
Disease
3347


In [330]:
for i in entity_pairs:
    print(i)
    axc=set()
    for j in names[i+'_all']:
        for k in j.get('PMID'):
            axc.add(k)
    print(len(axc))


Gene
26124
RNA
656
Protein
721
Carbohydrate
8012
Lipid
7380
Peptide
1587
Pharmaceutical_Preparations
148
Toxin
272
Mutation
1484
Disease
148697


In [345]:
all_es_result=json.load(open('results/step2/all_es_result.json'))

all_es_result=[]
for j in entity_pairs:
    for i in names[j+'_all']:
        if i.get('type')=='Disease':
            if i.get('url'):
                all_es_result.append({'entity':i.get('entity'),'PMID':i.get('PMID'),'target':i.get('target'),'date':i.get('date'),'type':i.get('type'),'sentence':i.get('sentence'),'_id':i.get('type')+'_'+i.get('entity')})
        elif i.get('type')=='Mutation':
            if i.get('Genomic View'):
                all_es_result.append({'entity':i.get('entity'),'PMID':i.get('PMID'),'target':i.get('target'),'date':i.get('date'),'type':i.get('type'),'sentence':i.get('sentence'),'_id':i.get('type')+'_'+i.get('entity')})
        else:
            all_es_result.append({'entity':i.get('entity'),'PMID':i.get('PMID'),'target':i.get('target'),'date':i.get('date'),'type':i.get('type'),'sentence':i.get('sentence'),'_id':i.get('type')+'_'+i.get('entity')})

In [348]:
for i in entity_pairs:
    names[i.replace(' ','_')+'_sentences']=set()
for i in all_es_result:
    for j in i.get('sentence'):
        for k in j:
            if re.search('RNA',i.get('type')):
                names['RNA'.replace(' ','_')+'_sentences'].add(k)
            else:
                names[i.get('type').replace(' ','_')+'_sentences'].add(k)


In [349]:

for i in entity_pairs:
    print(i)
    print(len(names[i.replace(' ','_')+'_sentences']))


Gene
78496
RNA
2029
Protein
1721
Carbohydrate
15724
Lipid
15182
Peptide
3114
Pharmaceutical_Preparations
238
Toxin
461
Mutation
3452
Disease
630059
